In [ ]:
# --- Imports ---
import os, json, math, io, zipfile, time, re, shutil, glob, pathlib, sys, subprocess, shlex, tempfile, warnings
from pathlib import Path
from datetime import datetime as dt, timedelta, timezone, date, datetime
from dateutil import parser as dateparser
from dateutil.relativedelta import relativedelta
from urllib.parse import urljoin, quote
from functools import reduce

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pytz
import xarray as xr
import rasterio
from rasterio.mask import mask

import shapely
from shapely import ops
from shapely.geometry import Point, Polygon, box, mapping
from shapely.ops import unary_union, transform as shp_transform

from pyproj import Transformer
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import folium
import ee  # Earth Engine

from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from geopy.distance import geodesic

import yaml

warnings.filterwarnings("ignore", category=UserWarning)


def skip_if_exists(path: str) -> bool:
    return os.path.exists(path)


In [ ]:
# --- Configuration loader (shared HeatShield/HydroPulse) ---
from pathlib import Path
import os
import re
import yaml

def load_env_file(path: Path) -> dict:
    env = {}
    if not path.exists():
        return env
    for line in path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, val = line.split("=", 1)
        env[key.strip()] = val.strip().strip('"').strip("'")
    return env

def apply_env_overrides() -> None:
    env = load_env_file(Path(".env"))
    for k, v in env.items():
        if v:
            os.environ[k] = v

def _fmt_yyyymmdd(s: str) -> str:
    # expects YYYY-MM-DD
    return re.sub(r"-", "", s.strip())

def _render_template(tpl: str, *, start: str, end: str, res_m: int, epsg: int, region: str) -> str:
    return tpl.format(
        start=_fmt_yyyymmdd(start),
        end=_fmt_yyyymmdd(end),
        res_m=int(res_m),
        epsg=int(epsg),
        region=str(region),
    )

def _resolve_out_dir(project_dir: Path, out_dir_value: str | None) -> Path:
    out_dir_value = out_dir_value or "results"
    p = Path(out_dir_value)
    return (p if p.is_absolute() else (project_dir / p)).resolve()

def _resolve_pathlike_keys(cfg: dict, out_dir: Path) -> None:
    """
    Resolve config values that look like relative file paths under out_dir.
    Rule: if key ends with _FILENAME, _PATH, _ZIP, _TIF_NAME, _CSV_NAME, _PARQUET_NAME, _TXT_NAME
    and the value is a relative path, make it absolute under out_dir.
    """
    suffixes = (
        "_FILENAME", "_PATH", "_ZIP",
        "_TIF_NAME", "_CSV_NAME", "_PARQUET_NAME", "_TXT_NAME",
        "_NETCDF_NAME", "_ZARR_NAME"
    )
    for k, v in list(cfg.items()):
        if not isinstance(v, str):
            continue
        if not k.endswith(suffixes):
            continue
        p = Path(v)
        if p.is_absolute():
            cfg[k] = str(p)
        else:
            cfg[k] = str((out_dir / p).resolve())

# Fail fast unless PROJECT_DIR is explicitly provided.
apply_env_overrides()

PROJECT_DIR = os.environ.get("PROJECT_DIR")
if not PROJECT_DIR:
    raise FileNotFoundError(
        "PROJECT_DIR is not set. Add PROJECT_DIR to .env or environment variables."
    )

PROJECT_DIR = Path(PROJECT_DIR).expanduser().resolve()
CONFIG_PATH = PROJECT_DIR / "config" / "config.yaml"
if not CONFIG_PATH.exists():
    raise FileNotFoundError(f"Missing config file: {CONFIG_PATH}")

CONFIG = yaml.safe_load(CONFIG_PATH.read_text()) or {}

# Set out_dir to an absolute path early.
OUT_DIR = _resolve_out_dir(PROJECT_DIR, CONFIG.get("out_dir", "results"))
CONFIG["out_dir"] = str(OUT_DIR)

# Optional: Apply env overrides ONLY for keys that exist in config.yaml.
# This prevents HeatShield-specific lists in shared code.
for key in list(CONFIG.keys()):
    env_val = os.environ.get(key)
    if env_val:
        CONFIG[key] = env_val

# Also allow a small, explicit allowlist for common optional overrides across projects.
for key in ["PURPLEAIR_SENSOR_INDEX"]:  # harmless if absent in HydroPulse config
    env_val = os.environ.get(key)
    if env_val:
        CONFIG[key] = env_val

# Render FINAL_DAILY_FILENAME from template if provided.
# Keeps downstream code stable: always refer to CONFIG["FINAL_DAILY_FILENAME"].
if "FINAL_DAILY_FILENAME_TEMPLATE" in CONFIG:
    region = CONFIG.get("region", "CA")
    start_date = CONFIG["start_date"]
    end_date = CONFIG["end_date"]
    res_m = CONFIG.get("grid_resolution_m", 3000)
    epsg = CONFIG.get("OPS_EPSG", CONFIG.get("CA_ALBERS_EPSG", 3310))
    CONFIG["FINAL_DAILY_FILENAME"] = _render_template(
        CONFIG["FINAL_DAILY_FILENAME_TEMPLATE"],
        start=start_date,
        end=end_date,
        res_m=res_m,
        epsg=epsg,
        region=region,
    )

# Resolve pathlike keys under out_dir (only for keys that exist).
_resolve_pathlike_keys(CONFIG, OUT_DIR)

# Create common directories only if they are referenced in config.
# (Avoid hardcoding "manual" for HydroPulse.)
for k, v in CONFIG.items():
    if k.endswith("_DIRNAME") and isinstance(v, str):
        os.makedirs(Path(CONFIG["out_dir"]) / v, exist_ok=True)

# EPSG constants (configurable)
WGS84_EPSG = int(CONFIG.get("WGS84_EPSG", 4326))
CA_ALBERS_EPSG = int(CONFIG.get("CA_ALBERS_EPSG", 3310))
OPS_EPSG = int(CONFIG.get("OPS_EPSG", CA_ALBERS_EPSG))

# Set working directory
os.chdir(PROJECT_DIR)

print(f"Config loaded from {CONFIG_PATH}")
print(f"Output dir: {CONFIG['out_dir']}")
print(f"Final daily filename: {CONFIG.get('FINAL_DAILY_FILENAME')}")

In [ ]:
from pathlib import Path

def resolve_out_path(path_str: str) -> str:
    """
    Resolve a path string relative to CONFIG['out_dir'] unless already absolute.
    Returns an absolute string path.
    """
    p = Path(path_str)
    if p.is_absolute():
        return str(p)
    return str((Path(CONFIG["out_dir"]) / p).resolve())

In [ ]:
# --- Ensure California boundary and build 3 km grid clipped to land ---


# Config
res_m = int(CONFIG.get("grid_resolution_m", 3000))
out_epsg = int(CONFIG.get("crs_epsg", 4326))
out_dir = CONFIG["out_dir"]; os.makedirs(out_dir, exist_ok=True)
inset_buffer_m = int(CONFIG.get("coast_inset_m", 0))  # e.g. 5000
boundary_path = CONFIG.get("ca_boundary_path", None)

# 1) Ensure boundary: download Census cartographic boundary if missing
if not boundary_path or not os.path.exists(boundary_path):
    states_zip = os.path.join(out_dir, "cb_2023_us_state_20m.zip")
    if not os.path.exists(states_zip):
        url = CONFIG["CENSUS_STATES_ZIP_URL"]
        r = requests.get(url, timeout=int(CONFIG.get("CENSUS_STATES_TIMEOUT", 120))); r.raise_for_status()
        with open(states_zip, "wb") as f: f.write(r.content)
    # Read from zip directly and select California
    states = gpd.read_file(f"zip://{states_zip}")
    if states.empty:
        raise ValueError("Census states file loaded empty.")
    ca = states[states["STATEFP"].astype(str).str.zfill(2).eq("06")][["geometry"]]
    if ca.empty:
        raise ValueError("California polygon not found in Census states file.")
    boundary_path = os.path.join(out_dir, "california_boundary.gpkg")
    ca.to_file(boundary_path, driver="GPKG")
    CONFIG["ca_boundary_path"] = boundary_path  # persist for later cells

# 2) Load boundary, dissolve, project, optional inward buffer
b = gpd.read_file(boundary_path)
if b.crs is None: raise ValueError("Boundary file has no CRS.")
b = b[["geometry"]].copy()
b = b.to_crs(CA_ALBERS_EPSG)
b = gpd.GeoDataFrame(geometry=[b.unary_union], crs=f"EPSG:{CA_ALBERS_EPSG}")
if inset_buffer_m > 0:
    b.geometry = b.buffer(-inset_buffer_m)
    b = gpd.GeoDataFrame(geometry=[b.unary_union], crs=f"EPSG:{CA_ALBERS_EPSG}")

# 3) Build snapped rectilinear grid over boundary bounds in EPSG:3310
minx, miny, maxx, maxy = b.total_bounds
snap_down = lambda v, s: np.floor(v/s)*s
snap_up   = lambda v, s: np.ceil(v/s)*s
minx, miny = snap_down(minx, res_m), snap_down(miny, res_m)
maxx, maxy = snap_up(maxx, res_m), snap_up(maxy, res_m)

xs = np.arange(minx, maxx, res_m)
ys = np.arange(miny, maxy, res_m)
n_rect = len(xs)*len(ys)
if n_rect > 3_500_000:
    raise MemoryError(f"Grid too large ({n_rect:,}). Increase res_m or tile the state.")

cells, col_i, row_j = [], [], []
for j, y in enumerate(ys):
    for i, x in enumerate(xs):
        cells.append(box(x, y, x+res_m, y+res_m)); col_i.append(i); row_j.append(j)

gdf_proj = gpd.GeoDataFrame({"col_i": np.int32(col_i), "row_j": np.int32(row_j)},
                            geometry=cells, crs=f"EPSG:{CA_ALBERS_EPSG}")
gdf_proj["cell_area_m2"] = float(res_m)*float(res_m)
gdf_proj["grid_id"] = f"CA3310_{res_m}_" + gdf_proj["col_i"].astype(str) + "_" + gdf_proj["row_j"].astype(str)

# 4) Strict land clip and land fraction
gdf_proj = gpd.sjoin(gdf_proj, b, how="inner", predicate="intersects").drop(columns=["index_right"])
inter = gpd.overlay(gdf_proj[["grid_id","geometry"]], b, how="intersection", keep_geom_type=True)
inter["land_area_m2"] = inter.geometry.area
land = inter[["grid_id","land_area_m2"]].groupby("grid_id", as_index=False).sum()
gdf_proj = gdf_proj.merge(land, on="grid_id", how="left")
gdf_proj["land_area_m2"] = gdf_proj["land_area_m2"].fillna(0.0)
gdf_proj["land_frac"] = (gdf_proj["land_area_m2"] / gdf_proj["cell_area_m2"]).clip(0,1)
gdf_proj = gdf_proj[gdf_proj["land_frac"] > 0].reset_index(drop=True)

# 5) Reproject to requested output CRS and save
grid_gdf = gdf_proj.to_crs(out_epsg)

parquet_path = os.path.join(out_dir, f"grid_{res_m}m_CA.parquet")
grid_gdf.to_parquet(parquet_path, index=False)

geojson_path = os.path.join(out_dir, f"grid_{res_m}m_CA_head10.geojson")
grid_gdf.head(10).to_file(geojson_path, driver="GeoJSON")

# Diagnostics
cell_area_km2 = (res_m/1000.0)**2
eff_land_km2 = float((grid_gdf.get("land_frac",1.0) * cell_area_km2).sum())
print(f"Saved: {parquet_path}")
print(f"Cells: {len(grid_gdf):,}")
print(f"Effective land area ≈ {round(eff_land_km2):,} km²")
print(f"Implied cell size ≈ {round((eff_land_km2/len(grid_gdf))**0.5,2)} km")

grid_gdf.head()


In [ ]:
# --- Persist config + save grid (3310 ops copy, 4326 preview) + write metadata ---

# Inputs assumed from prior cell:
# - grid_gdf            : current grid GeoDataFrame (any CRS)
# - CONFIG              : dict with out_dir, grid_resolution_m, crs_epsg, ca_boundary_path
# - CA_ALBERS_EPSG=3310 : defined earlier

out_dir = CONFIG["out_dir"]; os.makedirs(out_dir, exist_ok=True)
res_m = int(CONFIG.get("grid_resolution_m", 3000))
out_epsg = int(CONFIG.get("crs_epsg", 4326))
boundary_path = CONFIG.get("ca_boundary_path")

# 1) Persist boundary path back to CONFIG 
if not boundary_path or not os.path.exists(boundary_path):
    raise FileNotFoundError("CONFIG['ca_boundary_path'] missing or invalid. Rebuild boundary.")
CONFIG["ca_boundary_path"] = boundary_path

config_runtime_path = os.path.join(out_dir, "config_runtime.json")
with open(config_runtime_path, "w") as f:
    json.dump(CONFIG, f, indent=2)
print("Saved:", config_runtime_path)

# 2) Ensure we have an EPSG:3310 version for spatial ops
if grid_gdf.crs is None:
    raise ValueError("grid_gdf has no CRS. Rebuild grid.")
grid_3310 = grid_gdf.to_crs(3310) if grid_gdf.crs.to_epsg() != 3310 else grid_gdf

# 3) Save operational GeoParquet in 3310 + lightweight WGS84 preview
parquet_3310 = os.path.join(out_dir, f"grid_{res_m}m_CA_epsg3310.parquet")
grid_3310.to_parquet(parquet_3310, index=False)
print("Saved:", parquet_3310, "| cells:", len(grid_3310))

# Optional small preview in 4326 for quick map checks
preview_4326 = grid_3310.to_crs(4326).head(500)  # cap to avoid huge files
geojson_preview = os.path.join(out_dir, f"grid_{res_m}m_CA_head500_epsg4326.geojson")
preview_4326.to_file(geojson_preview, driver="GeoJSON")
print("Saved:", geojson_preview)

# 4) Compute and save metadata
cell_area_km2 = (res_m/1000.0)**2
effective_land_km2 = float((grid_3310.get("land_frac", 1.0) * cell_area_km2).sum())
implied_cell_km = float((effective_land_km2 / len(grid_3310))**0.5)
minx, miny, maxx, maxy = grid_3310.total_bounds
bbox_km = ((maxx-minx)/1000.0, (maxy-miny)/1000.0)

meta = {
    "timestamp_utc": dt.utcnow().isoformat(timespec="seconds") + "Z",
    "grid_resolution_m": res_m,
    "crs_ops_epsg": 3310,
    "crs_export_default_epsg": out_epsg,
    "cells": int(len(grid_3310)),
    "effective_land_area_km2": round(effective_land_km2, 2),
    "implied_cell_km": round(implied_cell_km, 4),
    "bbox_km_width_height": [round(bbox_km[0], 2), round(bbox_km[1], 2)],
    "has_land_frac": bool("land_frac" in grid_3310.columns),
    "boundary_path": boundary_path,
    "parquet_3310_path": parquet_3310,
    "geojson_preview_4326_path": geojson_preview,
}

meta_path = os.path.join(out_dir, f"grid_{res_m}m_CA_meta.json")
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)
print("Saved:", meta_path)
meta


In [ ]:
# CDO data fetch and processing functions
# repeat some variables for clarity
OUT_DIR = CONFIG["out_dir"]
RAW_DIR = os.path.join(OUT_DIR, CONFIG["CDO_RAW_DIRNAME"])
CLEAN_DIR = os.path.join(OUT_DIR, CONFIG["CDO_CLEAN_DIRNAME"])
os.makedirs(RAW_DIR, exist_ok=True); os.makedirs(CLEAN_DIR, exist_ok=True)

def month_windows(start_date, end_date):
    s = dt.fromisoformat(start_date).date().replace(day=1)
    e = dt.fromisoformat(end_date).date()
    cur = s
    while cur <= e:
        nxt = (cur + relativedelta(months=1)) - relativedelta(days=1)
        yield cur.isoformat(), min(nxt, e).isoformat()
        cur = (cur + relativedelta(months=1)).replace(day=1)

def parse_attributes(attr):
    parts = (attr or "").split(","); parts += [""] * (4 - len(parts))
    mflag, qflag, sflag, obs_hhmm = parts[:4]
    return mflag or None, qflag or None, sflag or None, obs_hhmm or None

def fetch_cdo_page(session, url, headers, params, max_retries=None, base_delay=None, timeout=None):
    if max_retries is None:
        max_retries = int(CONFIG.get("CDO_MAX_RETRIES", 6))
    if base_delay is None:
        base_delay = float(CONFIG.get("CDO_BACKOFF_BASE", 0.8))
    if timeout is None:
        timeout = int(CONFIG.get("CDO_TIMEOUT", 180))
    for attempt in range(max_retries):
        try:
            r = session.get(url, headers=headers, params=params, timeout=timeout)
            if r.status_code in (429, 500, 502, 503, 504):
                raise requests.HTTPError(f"{r.status_code} retry")
            r.raise_for_status()
            return r.json()
        except Exception:
            if attempt == max_retries - 1:
                raise
            time.sleep(base_delay * (2 ** attempt))


def cdo_stream_monthly(datasetid, locationid, startdate, enddate, datatypes, token,
                       units="standard", page_limit=1000, force=False):
    url = CONFIG["CDO_BASE_URL"]
    headers = {"token": token}
    session = requests.Session()
    written = []

    for dtid in datatypes:
        for ms, me in month_windows(startdate, enddate):
            out_csv = os.path.join(RAW_DIR, f"ghcnd_{dtid}_{ms[:7]}.csv")
            if skip_if_exists(out_csv) and not force:
                # resume: skip existing month-datatype file
                written.append(out_csv); continue

            frames = []
            offset = 1
            while True:
                params = {
                    "datasetid": datasetid, "locationid": locationid,
                    "startdate": ms, "enddate": me,
                    "datatypeid": dtid, "units": units,
                    "limit": page_limit, "offset": offset
                }
                js = fetch_cdo_page(session, url, headers, params)
                rows = js.get("results", [])
                if not rows:
                    break
                frames.append(pd.json_normalize(rows))
                if len(rows) < page_limit:
                    break
                offset += page_limit
                time.sleep(0.15)  # gentle pacing

            if frames:
                df = pd.concat(frames, ignore_index=True)
                # normalize
                df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
                parsed = df["attributes"].apply(parse_attributes)
                df[["mflag","qflag","sflag","obs_hhmm"]] = pd.DataFrame(parsed.tolist(), index=df.index)
                # scale tenths
                scale = {"PRCP": 0.1, "TMAX": 0.1, "TMIN": 0.1}
                df["datatype"] = df["datatype"].astype(str)
                df["value"] = pd.to_numeric(df["value"], errors="coerce")
                df["value_scaled"] = df.apply(lambda r: r["value"] * scale.get(r["datatype"], 1.0), axis=1)
                # write monthly raw
                df[["date","datatype","station","attributes","mflag","qflag","sflag","obs_hhmm","value","value_scaled"]].to_csv(out_csv, index=False)
                written.append(out_csv)
            else:
                # create an empty file with header to mark completion
                with open(out_csv, "w", newline="") as f:
                    w = csv.writer(f); w.writerow(["date","datatype","station","attributes","mflag","qflag","sflag","obs_hhmm","value","value_scaled"])
                written.append(out_csv)
    return written

def build_clean_wide():
    # read all monthly raw files and assemble cleaned wide once
    files = sorted([os.path.join(RAW_DIR, f) for f in os.listdir(RAW_DIR) if f.endswith(".csv")])
    if not files:
        return None
    df = pd.concat((pd.read_csv(f, dtype={"datatype":str,"station":str}) for f in files), ignore_index=True)
    # convert types back
    df["date"] = pd.to_datetime(df["date"]).dt.date
    # keep good qflag
    df = df[(df["qflag"].isna()) | (df["qflag"]=="")]
    wide = (
        df.pivot_table(index=["station","date"], columns="datatype", values="value_scaled", aggfunc="mean")
          .reset_index()
          .rename(columns={"date":"obs_date","PRCP":"precipitation_mm","TMAX":"temperature_max_c","TMIN":"temperature_min_c"})
          .sort_values(["obs_date","station"])
    )
    # attach obs time from PRCP
    prcp_times = df[df["datatype"]=="PRCP"][["station","date","obs_hhmm"]].drop_duplicates().rename(columns={"date":"obs_date"})
    wide = wide.merge(prcp_times, on=["station","obs_date"], how="left")
    raw_all = os.path.join(OUT_DIR, "ghcnd_daily_raw_all.csv")
    wide_all = os.path.join(OUT_DIR, "ghcnd_daily_wide.csv")
    df.to_csv(raw_all, index=False)
    wide.to_csv(wide_all, index=False)
    return raw_all, wide_all, len(df), len(wide), wide["station"].nunique(), wide["obs_date"].nunique()

# ---- Run statewide with resume capability ----
token = os.environ.get("CDO_TOKEN") or CONFIG.get("CDO_TOKEN", "")
if token and token != "YOUR_NCEI_CDO_TOKEN":
    written = cdo_stream_monthly(
        datasetid="GHCND",
        locationid="FIPS:06",                      # California statewide
        startdate=CONFIG["start_date"],
        enddate=CONFIG["end_date"],
        datatypes=["TMAX","TMIN","PRCP"],
        token=token,
        units="standard",
        page_limit=1000,
        force=False                                 # set True to re-download
    )
    print(f"Monthly files written: {len(written)} → {RAW_DIR}")

    res = build_clean_wide()
    if res:
        raw_all, wide_all, n_raw, n_wide, n_stn, n_dates = res
        print(f"Saved raw:  {raw_all}")
        print(f"Saved wide: {wide_all}")
        print(f"Counts → raw: {n_raw} | wide: {n_wide} | stations: {n_stn} | dates: {n_dates}")
else:
    print("Skipping CDO (missing CDO token).")


In [ ]:
# === GHCND DAILY: raw (long) -> cleaned (wide with lat/lon in bbox) ===
# Input  (from your earlier step):  results/ghcnd_daily_raw_all.csv  (long form)
# Output (used by superset):        results/ghcnd_daily_cleaned.parquet  (wide per station-day with lat/lon)

# Need to do this because we aren't getting proper "joins" in our superset setup.


BASE = CONFIG["out_dir"]
RAW = resolve_out_path(CONFIG["GHCND_RAW_CSV_NAME"])
OUT_PARQ = resolve_out_path(CONFIG["GHCND_CLEAN_PARQUET_NAME"])
OUT_CSV = resolve_out_path(CONFIG["GHCND_CLEAN_CSV_NAME"])

assert os.path.exists(RAW), f"Missing raw GHCND file: {RAW}"

# 1) Ensure we have a station catalog with lat/lon
#    Prefer a local copy if you already saved one; otherwise download NOAA's reference once.
CAT_DIR = os.path.join(BASE, CONFIG["MANUAL_DIRNAME"]); os.makedirs(CAT_DIR, exist_ok=True)
CAT_TXT = os.path.join(CAT_DIR, CONFIG["GHCND_STATIONS_TXT_NAME"])

if not os.path.exists(CAT_TXT):
    url = CONFIG["GHCND_STATIONS_URL"]
    r = requests.get(url, timeout=int(CONFIG.get("CENSUS_STATES_TIMEOUT", 120))); r.raise_for_status()
    with open(CAT_TXT, "wb") as f: f.write(r.content)

# Parse ghcnd-stations.txt (fixed-width)
# Columns per docs: ID(1-11), LAT(13-20), LON(22-30), ELEV(32-37), STATE(39-40), NAME(42-71) ...
def parse_stations(path):
    recs = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if len(line) < 40: 
                continue
            sid = line[0:11].strip()
            try:
                lat = float(line[12:20].strip())
                lon = float(line[21:30].strip())
            except ValueError:
                continue
            state = line[38:40].strip()
            name  = line[41:71].strip()
            recs.append((sid, lat, lon, state, name))
    return pd.DataFrame(recs, columns=["station_core","lat","lon","state","name"])

stations = parse_stations(CAT_TXT)

# 2) Load your raw long-form CDO file
# Expected columns seen in your sample:
# ['attributes','datatype','date','mflag','obs_hhmm','qflag','sflag','station','value','value_scaled']
raw = pd.read_csv(RAW, low_memory=False)

# Normalize station key: raw uses "GHCND:USW00023232" → core "USW00023232"
raw["station_core"] = raw["station"].astype(str).str.replace("^GHCND:", "", regex=True)

# Pick a numeric value column: prefer value_scaled if present; else scale GHCND native units.
# GHCND native: PRCP = tenths of mm, TMAX/TMIN = tenths of °C.
have_scaled = "value_scaled" in raw.columns
def scaled_val(row):
    if have_scaled and pd.notna(row["value_scaled"]):
        return float(row["value_scaled"])
    v = pd.to_numeric(row["value"], errors="coerce")
    if pd.isna(v): 
        return np.nan
    if row["datatype"] == "PRCP":
        return v * 0.1             # → mm
    if row["datatype"] in ("TMAX","TMIN"):
        return v * 0.1             # → °C
    return v

raw["val_clean"] = raw.apply(scaled_val, axis=1)

# Filter to the analysis window if your raw contains more than needed
if "start_date" in CONFIG and "end_date" in CONFIG:
    sd = pd.to_datetime(CONFIG["start_date"], utc=True, errors="coerce")
    ed = pd.to_datetime(CONFIG["end_date"],   utc=True, errors="coerce")
    raw["date"] = pd.to_datetime(raw["date"], utc=True, errors="coerce")
    raw = raw[(raw["date"]>=sd) & (raw["date"]<=ed)]
else:
    raw["date"] = pd.to_datetime(raw["date"], utc=True, errors="coerce")

# 3) Keep only the datatypes we need and one value per (station,date,datatype)
keep_types = {"PRCP":"precipitation_mm", "TMAX":"temperature_max_c", "TMIN":"temperature_min_c"}
raw = raw[raw["datatype"].isin(keep_types.keys())].copy()

# If multiple rows per (station,date,datatype), average them
agg = (raw.groupby(["station_core","date","datatype"], as_index=False)["val_clean"]
          .mean())

# 4) Pivot to wide columns
wide = (agg.pivot(index=["station_core","date"], columns="datatype", values="val_clean")
           .reset_index())
# Rename columns to our canonical names
wide = wide.rename(columns={k:v for k,v in keep_types.items() if k in wide.columns})

# 5) Attach lat/lon from station catalog and clip to CA bbox
wide = wide.merge(stations[["station_core","lat","lon"]], on="station_core", how="left")

# Clip to CONFIG["bbox"] (California in your setup)
bbox = CONFIG["bbox"]
minx, miny, maxx, maxy = bbox["nwlng"], bbox["selat"], bbox["selng"], bbox["nwlat"]
in_box = (wide["lon"].between(minx, maxx)) & (wide["lat"].between(miny, maxy))
wide = wide[in_box].copy()

# 6) Final tidy columns + sorts
cols_order = ["station_core","date","lat","lon",
              "precipitation_mm","temperature_max_c","temperature_min_c"]
for c in cols_order:
    if c not in wide.columns: wide[c] = np.nan
wide = wide[cols_order].sort_values(["station_core","date"])

# 7) Save for the superset
wide.to_parquet(OUT_PARQ, index=False)
wide.to_csv(OUT_CSV, index=False)
print(f"Saved cleaned CDO daily → {OUT_PARQ} (rows={len(wide)}, stations={wide['station_core'].nunique()})")


In [ ]:
### We've changed our approach -- DO NOT RUN THIS ANY MORE ###

# === PRISM | Cell A: Raw ingest via official Web Service (resumable + chunkable) ===
#
# Web service syntax (per PRISM doc):
#   https://services.nacse.org/prism/data/get/<region>/<res>/<element>/<date><?format=[nc|asc|bil]>
# One grid per request, returns a .zip.  [oai_citation:4‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
#
# PRISM download limits:
# - If a file is downloaded twice in a 24-hour period, no more downloads of that file allowed in that period
# - Excessive activity may result in IP blocking  [oai_citation:5‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
#
# This cell is designed for "download once, then reuse", and for running in small chunks.

import os
import time
import random
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime

# -----------------------------
# Required config (uses your existing baseline keys)
# -----------------------------
BASELINE_START = pd.to_datetime(CONFIG["BASELINE_START_DATE"])
BASELINE_END   = pd.to_datetime(CONFIG["BASELINE_END_DATE"])

# PRISM web service parameters
PRISM_SERVICE_BASE = str(CONFIG.get("PRISM_SERVICE_BASE_URL", "https://services.nacse.org/prism/data/get")) 
PRISM_REGION = str(CONFIG.get("PRISM_REGION", "us"))     # 'us' CONUS  [oai_citation:6‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
PRISM_RES    = str(CONFIG.get("PRISM_RESOLUTION", "4km"))# '4km' supported  [oai_citation:7‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
PRISM_ELEMENTS = CONFIG.get("PRISM_ELEMENTS", ["ppt", "tmean"])  # elements list in doc  [oai_citation:8‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)

# Output folder
OUT_DIR = Path(CONFIG["out_dir"])
RAW_DIR = OUT_DIR / "prism_raw_baseline_ws"
RAW_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------------
# Controls for "small chunks over several days"
# -----------------------------
# You can run year-by-year to reduce server load and make progress predictable.
# Set these each session (or add to config later if you like).
RUN_YEAR_START = int(CONFIG.get("PRISM_RUN_YEAR_START", BASELINE_START.year))
RUN_YEAR_END   = int(CONFIG.get("PRISM_RUN_YEAR_END", RUN_YEAR_START))  # default: single year
MAX_DOWNLOADS_THIS_RUN = int(CONFIG.get("PRISM_MAX_DOWNLOADS_PER_RUN", 400))  # hard stop per run

# Throttling/backoff
BASE_SLEEP_S = float(CONFIG.get("PRISM_BASE_SLEEP_S", 2.0))   # PRISM sample script uses sleep 2  [oai_citation:9‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
JITTER_S     = float(CONFIG.get("PRISM_JITTER_S", 0.75))
TIMEOUT_S    = int(CONFIG.get("PRISM_TIMEOUT_S", 120))

MAX_RETRIES  = int(CONFIG.get("PRISM_MAX_RETRIES", 6))
BACKOFF_BASE = float(CONFIG.get("PRISM_BACKOFF_BASE", 1.7))

# Optional: request format (default returns COG package; doc mentions optional formats nc/asc/bil)  [oai_citation:10‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
# Leave as None for default; or set to "bil" / "nc"
PRISM_FORMAT = CONFIG.get("PRISM_FORMAT", None)

# Optional: releaseDate check (defaults OFF).
# When OFF: "download once" behavior = if file exists, skip without checking.
USE_RELEASEDATE_CHECK = bool(CONFIG.get("PRISM_USE_RELEASEDATE_CHECK", False))

# -----------------------------
# URL helpers (per doc)
# -----------------------------
def prism_grid_url(element: str, yyyymmdd: str) -> str:
    # https://services.nacse.org/prism/data/get/<region>/<res>/<element>/<date><?format=...>  [oai_citation:11‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
    url = f"{PRISM_SERVICE_BASE}/{PRISM_REGION}/{PRISM_RES}/{element}/{yyyymmdd}"
    if PRISM_FORMAT:
        url += f"?format={PRISM_FORMAT}"
    return url

def prism_release_url(element: str, yyyymmdd: str) -> str:
    # https://services.nacse.org/prism/data/get/releaseDate/<region>/<resolution>/<element>/<date>?json=true  [oai_citation:12‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
    return f"{PRISM_SERVICE_BASE}/releaseDate/{PRISM_REGION}/{PRISM_RES}/{element}/{yyyymmdd}?json=true"

def safe_session() -> requests.Session:
    s = requests.Session()
    s.headers.update({
        "User-Agent": CONFIG.get("USER_AGENT_HEADERS", {}).get("User-Agent", "BlueLeafLabs/HydroPulse"),
        "Accept": "*/*",
    })
    return s

def parse_filename_from_cd(cd: str) -> str | None:
    # Content-Disposition: attachment; filename=prism_ppt_us_4km_19910101.zip
    if not cd:
        return None
    cd = cd.strip()
    parts = cd.split(";")
    for p in parts:
        p = p.strip()
        if p.lower().startswith("filename="):
            fn = p.split("=", 1)[1].strip().strip('"')
            return fn
    return None

# -----------------------------
# Local pathing
# -----------------------------
def out_path_for(element: str, yyyymmdd: str, filename_hint: str | None = None) -> Path:
    ed = RAW_DIR / element
    ed.mkdir(parents=True, exist_ok=True)
    if filename_hint:
        return ed / filename_hint
    # Fallback (stable and unique even if server changes naming slightly)
    suffix = PRISM_FORMAT if PRISM_FORMAT else "cog"
    return ed / f"prism_{element}_{PRISM_REGION}_{PRISM_RES}_{yyyymmdd}_{suffix}.zip"

# -----------------------------
# Optional releaseDate logic
# -----------------------------
def fetch_release_date(session: requests.Session, element: str, yyyymmdd: str) -> str | None:
    # doc: releaseDate service provides release date; older than Apr 2014 may be unpopulated  [oai_citation:13‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
    try:
        r = session.get(prism_release_url(element, yyyymmdd), timeout=TIMEOUT_S)
        if r.status_code != 200:
            return None
        js = r.json()
        # The PDF describes fields; response structure may be list/dict depending on single vs range.
        # We'll defensively extract any plausible release date string.
        if isinstance(js, list) and js:
            return js[0].get("releaseDate") or js[0].get("ReleaseDate") or js[0].get("release_date")
        if isinstance(js, dict):
            return js.get("releaseDate") or js.get("ReleaseDate") or js.get("release_date")
        return None
    except Exception:
        return None

# -----------------------------
# Download with retries + backoff
# -----------------------------
def download_one(session: requests.Session, element: str, yyyymmdd: str) -> tuple[str, Path | None]:
    url = prism_grid_url(element, yyyymmdd)

    # First request HEAD-like via GET stream (server returns a zip)
    for attempt in range(MAX_RETRIES):
        try:
            r = session.get(url, stream=True, timeout=TIMEOUT_S)
            if r.status_code == 404:
                return ("unavailable", None)
            if r.status_code in (429, 500, 502, 503, 504):
                raise RuntimeError(f"transient {r.status_code}")
            r.raise_for_status()

            fn = parse_filename_from_cd(r.headers.get("Content-Disposition", ""))
            out_path = out_path_for(element, yyyymmdd, fn)

            if out_path.exists():
                # Do not re-download. Close response promptly.
                r.close()
                return ("exists", out_path)

            tmp = out_path.with_suffix(out_path.suffix + ".part")
            with open(tmp, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):
                    if chunk:
                        f.write(chunk)
            os.replace(tmp, out_path)
            return ("downloaded", out_path)

        except Exception as e:
            # backoff + jitter
            sleep_s = (BACKOFF_BASE ** attempt) + random.random() * JITTER_S
            print(f"[WARN] {element} {yyyymmdd} attempt {attempt+1}/{MAX_RETRIES}: {e} -> sleep {sleep_s:.2f}s")
            time.sleep(sleep_s)

    return ("failed", None)

# -----------------------------
# Build run date range (year-sliced)
# -----------------------------
run_start = max(BASELINE_START, pd.Timestamp(year=RUN_YEAR_START, month=1, day=1))
run_end   = min(BASELINE_END,   pd.Timestamp(year=RUN_YEAR_END,   month=12, day=31))
dates = pd.date_range(run_start, run_end, freq="D")

print("PRISM Cell A (web service) starting")
print(f"Baseline window: {BASELINE_START.date()} → {BASELINE_END.date()}")
print(f"Run slice      : {run_start.date()} → {run_end.date()} ({len(dates)} days)")
print(f"Elements       : {PRISM_ELEMENTS}")
print(f"Resolution     : {PRISM_RES} | Region: {PRISM_REGION}")
print(f"Max downloads  : {MAX_DOWNLOADS_THIS_RUN}")
print(f"Sleep (on dl)  : {BASE_SLEEP_S}s + jitter up to {JITTER_S}s\n")

# -----------------------------
# Main loop
# -----------------------------
session = safe_session()

stats = {"downloaded": 0, "exists": 0, "unavailable": 0, "failed": 0, "skipped_release": 0}
downloads_this_run = 0

for element in PRISM_ELEMENTS:
    print(f"--- Element: {element} ---")
    for i, d in enumerate(dates, start=1):
        yyyymmdd = d.strftime("%Y%m%d")

        # Enforce per-run cap (lets you run small chunks over multiple days)
        if downloads_this_run >= MAX_DOWNLOADS_THIS_RUN:
            print(f"[STOP] Reached PRISM_MAX_DOWNLOADS_PER_RUN={MAX_DOWNLOADS_THIS_RUN}. Safe to rerun later.")
            break

        # If file exists, skip immediately (resume behavior)
        # We don’t know server filename until request, so check fallback name pattern too.
        # We’ll do a cheap existence check by globbing element dir for this date.
        el_dir = RAW_DIR / element
        if el_dir.exists():
            hits = list(el_dir.glob(f"*{yyyymmdd}*.zip"))
            if hits:
                stats["exists"] += 1
                continue

        # Optional release-date check (OFF by default)
        if USE_RELEASEDATE_CHECK:
            _ = fetch_release_date(session, element, yyyymmdd)  # you can wire this into a manifest later

        status, path = download_one(session, element, yyyymmdd)
        stats[status] += 1

        if status == "downloaded":
            downloads_this_run += 1
            # polite sleep only when we actually transfer bytes (PRISM sample script sleeps 2)  [oai_citation:14‡Prism Group](https://prism.oregonstate.edu/documents/PRISM_downloads_web_service.pdf)
            time.sleep(BASE_SLEEP_S + random.random() * JITTER_S)

        # Heartbeat every ~50 days
        if i % 50 == 0:
            print(f"{element} day {i}/{len(dates)} | dl={stats['downloaded']} exist={stats['exists']} unavail={stats['unavailable']} failed={stats['failed']}")

    print(f"Completed element: {element}\n")

print("PRISM Cell A complete (this run slice)")
print(stats)

In [ ]:
# === PRISM | Cell B: Build HydroPulse baseline from PRISM daily long-term normals (avg_30y) ===
#
# Inputs (local, no downloads):
#   {out_dir}/manual/prism/prism_ppt_us_25m_YYYYMMDD_avg_30y.zip
#   {out_dir}/manual/prism/prism_tmean_us_25m_YYYYMMDD_avg_30y.zip
#
# Output:
#   {out_dir}/prism_baseline_normals/prism_normals_doy_grid_25m_to_3km.parquet
#
# Output schema:
#   grid_id, doy, prism_ppt_norm_mm, prism_tmean_norm_c

import re
import zipfile
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio
from rasterio.io import MemoryFile
from pyproj import Transformer

OUT_DIR = Path(CONFIG["out_dir"])

# Where you placed PRISM normals
PRISM_DIR = Path(CONFIG.get("PRISM_MANUAL_DIR", OUT_DIR / "manual" / "prism"))
if not PRISM_DIR.exists():
    raise FileNotFoundError(f"PRISM_MANUAL_DIR not found: {PRISM_DIR}")

# HydroPulse grid (EPSG:3310)
OPS_EPSG = int(CONFIG.get("OPS_EPSG", 3310))
GRID_PATH = Path(resolve_out_path(CONFIG["GRID_FILENAME"]))

# Output
BASE_DIR = OUT_DIR / "prism_baseline_normals"
BASE_DIR.mkdir(parents=True, exist_ok=True)
SHARDS_DIR = BASE_DIR / "shards_doy"
SHARDS_DIR.mkdir(parents=True, exist_ok=True)
FINAL_PATH = BASE_DIR / "prism_normals_doy_grid_25m_to_3km.parquet"

# --- Load grid centroids ---
grid = gpd.read_parquet(GRID_PATH)
if grid.crs is None or (grid.crs.to_epsg() or 0) != OPS_EPSG:
    grid = grid.set_crs(f"EPSG:{OPS_EPSG}", allow_override=True) if grid.crs is None else grid.to_crs(OPS_EPSG)

if "grid_id" not in grid.columns:
    raise KeyError("Expected grid parquet to contain 'grid_id'.")

grid_id = grid["grid_id"].astype(str).values
centroids = grid.geometry.centroid
xs = centroids.x.values
ys = centroids.y.values
n_cells = len(grid_id)

print(f"Grid: {n_cells} cells | EPSG:{OPS_EPSG}")
print(f"PRISM normals dir: {PRISM_DIR}")

# --- PRISM filename parser (your observed convention) ---
# Example: prism_ppt_us_25m_20200115_avg_30y.zip
pat = re.compile(r"^prism_(ppt|tmean)_us_25m_(\d{8})_avg_30y\.zip$", re.IGNORECASE)

ppt = {}
tmean = {}

for p in sorted(PRISM_DIR.glob("*.zip")):
    m = pat.match(p.name)
    if not m:
        continue
    var = m.group(1).lower()
    yyyymmdd = m.group(2)

    # PRISM daily normals commonly use year=2020 as a convenient leap-year index;
    # we convert YYYYMMDD -> DOY using that year token directly.
    dt = pd.to_datetime(yyyymmdd, format="%Y%m%d", utc=True)
    doy = int(dt.dayofyear)

    if var == "ppt":
        ppt[doy] = p
    elif var == "tmean":
        tmean[doy] = p

doys = sorted(set(ppt.keys()) & set(tmean.keys()))
if not doys:
    raise RuntimeError(
        "No matching PRISM ppt/tmean normals found.\n"
        f"Expected filenames like: prism_ppt_us_25m_20200115_avg_30y.zip in {PRISM_DIR}"
    )

print(f"Matched DOYs: {len(doys)} (e.g., {doys[:5]})")

# --- Read GeoTIFF inside PRISM zip via MemoryFile ---
def open_tif_from_zip(zip_path: Path):
    with zipfile.ZipFile(zip_path, "r") as z:
        tif_names = [n for n in z.namelist() if n.lower().endswith(".tif")]
        if not tif_names:
            raise FileNotFoundError(f"No .tif found inside {zip_path.name}")
        tif_name = tif_names[0]
        tif_bytes = z.read(tif_name)

    mem = MemoryFile(tif_bytes)
    ds = mem.open()
    return mem, ds

def sample_zip_to_grid(zip_path: Path, xs3310: np.ndarray, ys3310: np.ndarray) -> np.ndarray:
    mem, ds = open_tif_from_zip(zip_path)
    try:
        tf = Transformer.from_crs(f"EPSG:{OPS_EPSG}", ds.crs, always_xy=True)
        sx, sy = tf.transform(xs3310, ys3310)

        vals = np.array([v[0] for v in ds.sample(zip(sx, sy))], dtype=np.float64)
        nodata = ds.nodata
        if nodata is not None:
            vals[vals == nodata] = np.nan
        vals[~np.isfinite(vals)] = np.nan
        return vals
    finally:
        ds.close()
        mem.close()

# --- Build shards per DOY (resumable) ---
written = 0
skipped = 0

for doy in doys:
    shard_path = SHARDS_DIR / f"prism_normals_doy_{doy:03d}.parquet"
    if shard_path.exists():
        skipped += 1
        continue

    ppt_vals = sample_zip_to_grid(ppt[doy], xs, ys).astype(np.float32)      # mm
    tm_vals  = sample_zip_to_grid(tmean[doy], xs, ys).astype(np.float32)    # °C

    out = pd.DataFrame({
        "grid_id": grid_id,
        "doy": np.full(n_cells, doy, dtype=np.int16),
        "prism_ppt_norm_mm": ppt_vals,
        "prism_tmean_norm_c": tm_vals,
    })
    out.to_parquet(shard_path, index=False)
    written += 1

    if written % 25 == 0:
        print(f"Shards written: {written} | latest DOY={doy:03d}")

print(f"Shard pass complete | written={written} | skipped={skipped}")

# --- Combine to one baseline parquet (fast enough at 366 shards) ---
shards = sorted(SHARDS_DIR.glob("prism_normals_doy_*.parquet"))
df_all = pd.concat((pd.read_parquet(p) for p in shards), ignore_index=True)
df_all.to_parquet(FINAL_PATH, index=False)

print(f"Saved baseline: {FINAL_PATH}")
print(f"Rows: {len(df_all)} | doys: {df_all['doy'].nunique()} | cells: {df_all['grid_id'].nunique()}")
print(df_all.head())

In [ ]:
# === SMAP (SPL4SMGP) via Harmony: CA-only subset + quiet + strong resume ===

from pathlib import Path
import os
import datetime as dt

import earthaccess
from harmony import BBox, Client, Collection, Request, CapabilitiesRequest

# ---------- 0) env reload (reuse your existing helpers if available) ----------
try:
    apply_env_overrides()
except NameError:
    pass

if not os.environ.get("EARTHDATA_TOKEN") and not (
    os.environ.get("EARTHDATA_USERNAME") and os.environ.get("EARTHDATA_PASSWORD")
):
    raise RuntimeError("Missing Earthdata credentials. Set EARTHDATA_TOKEN (recommended) in .env.")

earthaccess.login(strategy="environment")  # should be quiet if already logged in

# ---------- 1) config ----------
bbox = CONFIG["bbox"]
W = float(bbox["nwlng"])
S = float(bbox["selat"])
E = float(bbox["selng"])
N = float(bbox["nwlat"])

START = dt.date.fromisoformat(CONFIG["start_date"])
END   = dt.date.fromisoformat(CONFIG["end_date"])

SHORT_NAME = CONFIG.get("SMAP_L4_SHORT_NAME", "SPL4SMGP")
DESIRED_VARS = CONFIG.get("SMAP_L4_VARIABLES", ["sm_surface", "sm_rootzone"])

OUT_DIR = Path(CONFIG["out_dir"])
SMAP_DIR = OUT_DIR / "manual" / "smap" / SHORT_NAME
SMAP_DIR.mkdir(parents=True, exist_ok=True)

SENTINEL_DIR = SMAP_DIR / "_done"
SENTINEL_DIR.mkdir(parents=True, exist_ok=True)

def month_start(d: dt.date) -> dt.date:
    return dt.date(d.year, d.month, 1)

def next_month(d: dt.date) -> dt.date:
    return dt.date(d.year + (d.month == 12), 1 if d.month == 12 else d.month + 1, 1)

def month_range(start: dt.date, end: dt.date):
    cur = month_start(start)
    while cur <= end:
        nxt = next_month(cur)
        yield cur, min(end + dt.timedelta(days=1), nxt)  # end is effectively exclusive
        cur = nxt

def has_any_files(folder: Path) -> bool:
    if not folder.exists():
        return False
    # Harmony may write nested outputs; look recursively for non-empty files
    for p in folder.rglob("*"):
        if p.is_file() and p.stat().st_size > 0:
            return True
    return False

print("SMAP/Harmony (quiet) setup")
print("  short_name:", SHORT_NAME)
print("  bbox (W,S,E,N):", (W, S, E, N))
print("  time:", START, "→", END)
print("  root:", SMAP_DIR)

# ---------- 2) capabilities + variable sanitization ----------
harmony_client = Client()

cap_req = CapabilitiesRequest(short_name=SHORT_NAME)
cap = harmony_client.submit(cap_req)

import json
if isinstance(cap, str):
    cap = json.loads(cap)

concept_id = cap.get("conceptId") or cap.get("concept_id") or cap.get("conceptID")
if not concept_id:
    raise RuntimeError(f"Could not determine conceptId for {SHORT_NAME} from Harmony capabilities.")

available_var_names = set()
for v in (cap.get("variables") or []):
    if isinstance(v, dict) and "name" in v:
        available_var_names.add(v["name"])

if available_var_names:
    chosen = []
    for name in DESIRED_VARS:
        if name in available_var_names:
            chosen.append(name)
        else:
            hits = [vn for vn in available_var_names if vn == name or vn.endswith(name) or name in vn]
            if hits:
                chosen.append(hits[0])
    DESIRED_VARS = sorted(set(chosen))

# ---------- 3) submit/download month-by-month (quiet + strong resume) ----------
jobs_submitted = 0
months_skipped = 0
files_downloaded = 0

for m0, m1 in month_range(START, END):
    tag = f"{m0:%Y%m}"
    sentinel = SENTINEL_DIR / f"{tag}.done"
    month_dir = SMAP_DIR / tag

    # Strong resume: skip if sentinel exists OR month folder already has files
    if sentinel.exists() or has_any_files(month_dir):
        months_skipped += 1
        if not sentinel.exists():
            # create sentinel so future runs are clean
            sentinel.write_text("done=1\nnote=folder already had files\n")
        print(f"[SKIP] {tag}")
        continue

    req = Request(
        collection=Collection(id=concept_id),
        spatial=BBox(W, S, E, N),
        temporal={"start": dt.datetime(m0.year, m0.month, m0.day),
                  "stop":  dt.datetime(m1.year, m1.month, m1.day)},
        variables=DESIRED_VARS if DESIRED_VARS else None,
    )

    if not req.is_valid():
        raise RuntimeError(f"Invalid Harmony request for month {tag}: check bbox/time/vars")

    print(f"[RUN] {tag} submitting…")
    job_id = harmony_client.submit(req)
    jobs_submitted += 1

    # Quiet wait (no progress spam)
    harmony_client.wait_for_processing(job_id, show_progress=False)

    month_dir.mkdir(parents=True, exist_ok=True)

    futures = harmony_client.download_all(job_id, directory=str(month_dir), overwrite=False)
    out_files = [f.result() for f in futures]
    out_files = [p for p in out_files if p]

    files_downloaded += len(out_files)
    sentinel.write_text(f"job_id={job_id}\nfiles={len(out_files)}\n")
    print(f"[DONE] {tag} files={len(out_files)}")

print("SMAP/Harmony complete")
print("  jobs submitted:", jobs_submitted)
print("  months skipped:", months_skipped)
print("  files downloaded this run:", files_downloaded)
print("  root dir:", SMAP_DIR)

In [ ]:
# === SMAP Cell A (fixed): SPL4SMGP Harmony subset -> daily canonical parquet (resume-safe, quiet) ===

from pathlib import Path
import os
import re
import numpy as np
import pandas as pd
import xarray as xr

OUT_DIR = Path(CONFIG["out_dir"])
SMAP_ROOT = OUT_DIR / Path(CONFIG.get("SMAP_L4_DIRNAME", "manual/smap/SPL4SMGP"))
if not SMAP_ROOT.exists():
    raise FileNotFoundError(f"Missing SMAP root dir: {SMAP_ROOT}")

DAILY_DIR = OUT_DIR / Path(CONFIG.get("SMAP_DAILY_DIRNAME", "derived/smap_daily"))
DAILY_DIR.mkdir(parents=True, exist_ok=True)

GROUP = (CONFIG.get("SMAP_NETCDF_GROUP", "Geophysical_Data") or "").strip()
VAR_SURF = CONFIG.get("SMAP_VAR_SURFACE", "sm_surface")
VAR_ROOT = CONFIG.get("SMAP_VAR_ROOTZONE", "sm_rootzone")

TEMPLATE = CONFIG.get("SMAP_DAILY_TEMPLATE", "smap_daily_{date}.parquet")
AGG = (CONFIG.get("SMAP_DAILY_AGG", "median") or "median").lower()
LOG_EVERY = int(CONFIG.get("SMAP_LOG_EVERY_N_DAYS", 20))

if AGG not in {"median", "mean"}:
    raise ValueError("SMAP_DAILY_AGG must be 'median' or 'mean'")

# Filenames like: SMAP_L4_SM_gph_20240731T223000_Vv8010_001_subsetted.nc4
TS_RE = re.compile(r"_(\d{8})T(\d{6})_")

def parse_dt_from_name(p: Path) -> pd.Timestamp:
    m = TS_RE.search(p.name)
    if not m:
        raise ValueError(f"Cannot parse timestamp from filename: {p.name}")
    ymd, hms = m.group(1), m.group(2)
    return pd.Timestamp(f"{ymd}{hms}", tz="UTC")

def out_path_for_date(d_utc: pd.Timestamp) -> Path:
    return DAILY_DIR / TEMPLATE.format(date=d_utc.date().isoformat())

def open_smap(fp: Path) -> xr.Dataset:
    # Use netcdf4 engine; open the science group
    return xr.open_dataset(fp, engine="netcdf4", group=GROUP, decode_times=False, mask_and_scale=True)

def agg_stack(stack: np.ndarray) -> np.ndarray:
    # stack shape: (T, H, W), with NaN
    if AGG == "median":
        with np.errstate(all="ignore"):
            return np.nanmedian(stack, axis=0)
    return np.nanmean(stack, axis=0)

# Collect likely data files (Harmony outputs often .nc4)
OK_EXT = {".nc4", ".nc", ".cdf", ".h5", ".hdf5"}
all_files = sorted(
    p for p in SMAP_ROOT.rglob("*")
    if p.is_file()
    and "_done" not in p.parts
    and not p.name.endswith(".done")
    and p.stat().st_size > 0
    and (p.suffix.lower() in OK_EXT)
)

if not all_files:
    raise FileNotFoundError(f"No SMAP data files found under {SMAP_ROOT}")

# Build index of files with timestamps
rows = []
for fp in all_files:
    try:
        ts = parse_dt_from_name(fp)
    except Exception:
        continue
    rows.append((fp, ts, ts.normalize()))

idx = pd.DataFrame(rows, columns=["path", "ts_utc", "date_utc"])
if idx.empty:
    raise RuntimeError("Found SMAP files but none matched the expected timestamp pattern.")

# Filter to your configured analysis window
start = pd.Timestamp(CONFIG["start_date"], tz="UTC")
end = pd.Timestamp(CONFIG["end_date"], tz="UTC") + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)
idx = idx[(idx["ts_utc"] >= start) & (idx["ts_utc"] <= end)].copy()
idx.sort_values(["date_utc", "ts_utc"], inplace=True)

dates = idx["date_utc"].drop_duplicates().tolist()
if not dates:
    raise RuntimeError("No SMAP files within CONFIG start/end window.")

print("SMAP Cell A starting")
print("  SMAP root      :", SMAP_ROOT)
print("  Files in window:", len(idx))
print("  Days in window :", len(dates))
print("  Group/vars     :", GROUP, VAR_SURF, VAR_ROOT)
print("  Output dir     :", DAILY_DIR)

processed = 0
skipped = 0

for i, d in enumerate(dates, start=1):
    outp = out_path_for_date(d)
    if outp.exists() and outp.stat().st_size > 0:
        skipped += 1
        continue

    day_files = idx.loc[idx["date_utc"] == d, "path"].tolist()
    if not day_files:
        continue

    surf_list, root_list = [], []

    for fp in day_files:
        ds = open_smap(fp)

        # pull arrays
        surf = ds[VAR_SURF].astype("float32").values
        root = ds[VAR_ROOT].astype("float32").values

        # replace fill (-9999) + enforce plausible range [0,1]
        surf = np.where((surf >= 0.0) & (surf <= 1.0), surf, np.nan)
        root = np.where((root >= 0.0) & (root <= 1.0), root, np.nan)

        surf_list.append(surf)
        root_list.append(root)

        ds.close()

    surf_stack = np.stack(surf_list, axis=0)
    root_stack = np.stack(root_list, axis=0)

    surf_day = agg_stack(surf_stack)
    root_day = agg_stack(root_stack)

    n_obs = np.sum(np.isfinite(surf_stack), axis=0).astype("int16")

    H, W = surf_day.shape
    yy, xx = np.indices((H, W))

    df = pd.DataFrame({
        "date_utc": np.repeat(d, H * W),
        "y": yy.ravel().astype("int32"),
        "x": xx.ravel().astype("int32"),
        "sm_surface": surf_day.ravel().astype("float32"),
        "sm_rootzone": root_day.ravel().astype("float32"),
        "n_obs": n_obs.ravel().astype("int16"),
    })

    # Drop pixels with no data at all
    df = df[~(df["sm_surface"].isna() & df["sm_rootzone"].isna())]

    tmp = outp.with_suffix(".parquet.tmp")
    df.to_parquet(tmp, index=False)
    os.replace(tmp, outp)

    processed += 1
    if (processed % LOG_EVERY) == 0 or i == len(dates):
        print(f"  day {i}/{len(dates)} | wrote={processed} skipped={skipped} | last={d.date()} | rows={len(df):,}")

print("SMAP Cell A complete")
print("  wrote  :", processed)
print("  skipped:", skipped)
print("  daily dir:", DAILY_DIR)

In [ ]:
# Some SMAP validation/QA on daily parquet outputs

from pathlib import Path
import pandas as pd
import numpy as np

OUT_DIR = Path(CONFIG["out_dir"])
DAILY_DIR = OUT_DIR / Path(CONFIG.get("SMAP_DAILY_DIRNAME", "derived/smap_daily"))

files = sorted(DAILY_DIR.glob("smap_daily_*.parquet"))
assert files, f"No SMAP daily parquet files found in {DAILY_DIR}"

# Sample a few days + last day
sample_files = [files[0], files[len(files)//2], files[-1]]
rows = []

for fp in sample_files:
    df = pd.read_parquet(fp)

    rows.append({
        "file": fp.name,
        "rows": len(df),
        "unique_pixels": df[["y","x"]].drop_duplicates().shape[0],
        "sm_surface_min": float(np.nanmin(df["sm_surface"].values)) if df["sm_surface"].notna().any() else np.nan,
        "sm_surface_p01": float(np.nanpercentile(df["sm_surface"].dropna(), 1)) if df["sm_surface"].notna().any() else np.nan,
        "sm_surface_p50": float(np.nanpercentile(df["sm_surface"].dropna(), 50)) if df["sm_surface"].notna().any() else np.nan,
        "sm_surface_p99": float(np.nanpercentile(df["sm_surface"].dropna(), 99)) if df["sm_surface"].notna().any() else np.nan,
        "sm_surface_max": float(np.nanmax(df["sm_surface"].values)) if df["sm_surface"].notna().any() else np.nan,
        "sm_rootzone_p50": float(np.nanpercentile(df["sm_rootzone"].dropna(), 50)) if df["sm_rootzone"].notna().any() else np.nan,
        "n_obs_p50": float(np.nanpercentile(df["n_obs"].values, 50)),
        "n_obs_min": int(df["n_obs"].min()),
        "n_obs_max": int(df["n_obs"].max()),
        "nan_surface_frac": float(df["sm_surface"].isna().mean()),
        "nan_rootzone_frac": float(df["sm_rootzone"].isna().mean()),
    })

qa = pd.DataFrame(rows)
qa

In [ ]:
# === SMAP Cell B: daily SMAP pixels -> HydroPulse 3km grid_id daily table (resume-safe) ===

from pathlib import Path
import os
import numpy as np
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
import xarray as xr

OUT_DIR = Path(CONFIG["out_dir"])

# Inputs
GRID_PATH = Path(resolve_out_path(CONFIG.get("GRID_FILENAME", "grid_3000m_CA_epsg3310.parquet")))
SMAP_ROOT = OUT_DIR / Path(CONFIG.get("SMAP_L4_DIRNAME", "manual/smap/SPL4SMGP"))
SMAP_DAILY_DIR = OUT_DIR / Path(CONFIG.get("SMAP_DAILY_DIRNAME", "derived/smap_daily"))

if not GRID_PATH.exists():
    raise FileNotFoundError(f"Missing grid parquet: {GRID_PATH}")
if not SMAP_ROOT.exists():
    raise FileNotFoundError(f"Missing SMAP root: {SMAP_ROOT}")
if not SMAP_DAILY_DIR.exists():
    raise FileNotFoundError(f"Missing SMAP daily dir: {SMAP_DAILY_DIR}")

# Outputs
GRIDMAP_PATH = OUT_DIR / CONFIG.get("SMAP_GRIDMAP_FILENAME", "smap_pixel_to_grid_3310.parquet")
SHARDS_DIR = OUT_DIR / Path(CONFIG.get("SMAP_GRID_DAILY_SHARDS_DIRNAME", "derived/smap_grid_shards"))
SHARDS_DIR.mkdir(parents=True, exist_ok=True)

final_name_tmpl = CONFIG.get("SMAP_GRID_FINAL_FILENAME", "smap_daily_grid_CA_3000m_epsg3310_{start}_{end}.parquet")
FINAL_PATH = OUT_DIR / final_name_tmpl.format(
    start=CONFIG["start_date"].replace("-", ""),
    end=CONFIG["end_date"].replace("-", "")
)

AGG = (CONFIG.get("SMAP_GRID_AGG", "mean") or "mean").lower()
MIN_PIX = int(CONFIG.get("SMAP_GRID_MIN_PIXELS", 1))
LOG_EVERY = int(CONFIG.get("SMAP_LOG_EVERY_N_DAYS", 20))
NEAR_KM = float(CONFIG.get("SMAP_NEAREST_FALLBACK_KM", 6.0))
NEAR_M = NEAR_KM * 1000.0

if AGG not in {"mean", "median"}:
    raise ValueError("SMAP_GRID_AGG must be 'mean' or 'median'")

# -------------------------
# 1) Load HydroPulse grid
# -------------------------
grid = gpd.read_parquet(GRID_PATH)
if "grid_id" not in grid.columns:
    raise KeyError("Grid parquet must contain 'grid_id' column.")
if "geometry" not in grid.columns:
    raise KeyError("Grid parquet must contain 'geometry' column.")

# Ensure CRS is EPSG:3310 (your OPS_EPSG)
OPS_EPSG = int(CONFIG.get("OPS_EPSG", 3310))
if grid.crs is None:
    grid = grid.set_crs(epsg=OPS_EPSG)
else:
    grid = grid.to_crs(epsg=OPS_EPSG)

grid = grid[["grid_id", "geometry"]].copy()

# -------------------------
# 2) Build (y,x)->grid_id map (one-time), using cell_lat/cell_lon from a sample .nc4
# -------------------------
def find_sample_nc4() -> Path:
    ok_ext = {".nc4", ".nc", ".cdf"}
    cands = sorted(
        p for p in SMAP_ROOT.rglob("*")
        if p.is_file()
        and "_done" not in p.parts
        and p.stat().st_size > 0
        and p.suffix.lower() in ok_ext
    )
    if not cands:
        raise FileNotFoundError(f"No .nc/.nc4 files found under {SMAP_ROOT}")
    # pick largest, tends to be real science granule
    return max(cands, key=lambda p: p.stat().st_size)

def build_gridmap() -> pd.DataFrame:
    sample = find_sample_nc4()

    # Root group contains cell_lat/cell_lon per your earlier debug
    ds_root = xr.open_dataset(sample, engine="netcdf4", decode_times=False, mask_and_scale=True)

    # Expect 2D arrays named cell_lat / cell_lon
    if "cell_lat" not in ds_root.variables or "cell_lon" not in ds_root.variables:
        raise KeyError(f"Expected cell_lat/cell_lon in root group. Vars: {list(ds_root.variables.keys())[:50]}")

    lat = ds_root["cell_lat"].values
    lon = ds_root["cell_lon"].values

    # y/x coordinate arrays exist too; but we use integer indices y,x matching your daily parquet
    H, W = lat.shape
    yy, xx = np.indices((H, W))

    # Flatten
    df = pd.DataFrame({
        "y": yy.ravel().astype("int32"),
        "x": xx.ravel().astype("int32"),
        "lat": lat.ravel().astype("float64"),
        "lon": lon.ravel().astype("float64"),
    })
    ds_root.close()

    # Drop any missing or insane coords
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["lat", "lon"])
    df = df[(df["lat"] >= -90) & (df["lat"] <= 90) & (df["lon"] >= -180) & (df["lon"] <= 180)]

    # Make points in WGS84 then project to 3310
    gdf = gpd.GeoDataFrame(
        df[["y", "x"]].copy(),
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326"
    ).to_crs(epsg=OPS_EPSG)

    # Spatial join to grid polygons
    joined = gpd.sjoin(gdf, grid, how="left", predicate="within")[["y", "x", "grid_id", "geometry"]].copy()

    missing = joined["grid_id"].isna().sum()
    if missing > 0:
        # Fallback: nearest join within radius (meters)
        # Keep only missing points for nearest join
        miss = joined[joined["grid_id"].isna()].drop(columns=["grid_id"]).copy()
        # sjoin_nearest is available in geopandas >=0.10; use max_distance to avoid nonsense matches
        nearest = gpd.sjoin_nearest(miss, grid, how="left", max_distance=NEAR_M, distance_col="dist_m")
        joined.loc[joined["grid_id"].isna(), "grid_id"] = nearest["grid_id"].values

    joined = joined.drop(columns=["geometry"])
    joined = joined.dropna(subset=["grid_id"]).copy()

    # Enforce uniqueness
    joined = joined.drop_duplicates(subset=["y", "x"])
    return joined

if GRIDMAP_PATH.exists() and GRIDMAP_PATH.stat().st_size > 0:
    gridmap = pd.read_parquet(GRIDMAP_PATH)
else:
    gridmap = build_gridmap()
    tmp = GRIDMAP_PATH.with_suffix(".tmp.parquet")
    gridmap.to_parquet(tmp, index=False)
    os.replace(tmp, GRIDMAP_PATH)

if not {"y","x","grid_id"}.issubset(set(gridmap.columns)):
    raise RuntimeError(f"Bad gridmap schema: {gridmap.columns}")

print("SMAP Cell B starting")
print("  Grid:", GRID_PATH)
print("  SMAP daily dir:", SMAP_DAILY_DIR)
print("  Gridmap:", GRIDMAP_PATH, f"(rows={len(gridmap):,})")
print("  Shards dir:", SHARDS_DIR)
print("  FINAL:", FINAL_PATH)

# -------------------------
# 3) Process day shards (resume-safe)
# -------------------------
daily_files = sorted(SMAP_DAILY_DIR.glob("smap_daily_*.parquet"))
if not daily_files:
    raise FileNotFoundError(f"No smap_daily parquet files found in {SMAP_DAILY_DIR}")

written = 0
skipped = 0

def agg_group(df: pd.DataFrame) -> pd.DataFrame:
    # df has grid_id and moisture fields
    if AGG == "mean":
        out = df.groupby("grid_id", as_index=False).agg(
            sm_surface=("sm_surface", "mean"),
            sm_rootzone=("sm_rootzone", "mean"),
            n_pixels=("grid_id", "size"),
            n_obs_mean=("n_obs", "mean"),
        )
    else:
        out = df.groupby("grid_id", as_index=False).agg(
            sm_surface=("sm_surface", "median"),
            sm_rootzone=("sm_rootzone", "median"),
            n_pixels=("grid_id", "size"),
            n_obs_mean=("n_obs", "mean"),
        )
    return out

for i, fp in enumerate(daily_files, start=1):
    date_str = fp.stem.split("_")[-1]  # YYYY-MM-DD
    outp = SHARDS_DIR / f"smap_grid_{date_str}.parquet"

    if outp.exists() and outp.stat().st_size > 0:
        skipped += 1
        continue

    df = pd.read_parquet(fp)  # columns: date_utc,y,x,sm_surface,sm_rootzone,n_obs
    # Join mapping
    df = df.merge(gridmap, on=["y","x"], how="inner")

    # Aggregate to grid
    g = agg_group(df)

    # Apply min pixels filter
    g = g[g["n_pixels"] >= MIN_PIX].copy()
    g.insert(1, "date_utc", pd.Timestamp(date_str, tz="UTC"))

    tmp = outp.with_suffix(".tmp.parquet")
    g.to_parquet(tmp, index=False)
    os.replace(tmp, outp)

    written += 1
    if (written % LOG_EVERY) == 0 or i == len(daily_files):
        print(f"  day {i}/{len(daily_files)} | wrote={written} skipped={skipped} | last={date_str} | rows={len(g):,}")

print("SMAP Cell B shards complete")
print("  wrote  :", written)
print("  skipped:", skipped)

# -------------------------
# 4) Optional: stitch shards into one final parquet (resume-safe)
# -------------------------
if FINAL_PATH.exists() and FINAL_PATH.stat().st_size > 0:
    print("[SKIP] Final exists:", FINAL_PATH)
else:
    shard_files = sorted(SHARDS_DIR.glob("smap_grid_*.parquet"))
    if not shard_files:
        raise RuntimeError("No SMAP grid shards found to stitch.")
    parts = [pd.read_parquet(p) for p in shard_files]
    final = pd.concat(parts, ignore_index=True)
    tmp = FINAL_PATH.with_suffix(".tmp.parquet")
    final.to_parquet(tmp, index=False)
    os.replace(tmp, FINAL_PATH)
    print("Saved FINAL:", FINAL_PATH, "rows=", len(final))

In [ ]:
# ============================
# SNOTEL — Cell A
# Parse, normalize, and sanity-check
# First, we download data manually from https://wcc.sc.egov.usda.gov/reportGenerator/
# Parameters are: click on "Advanced search", select Network - SNOTEL, all stations in CA - this is about 500 stations,
# date range 1991-01-01 to 2026-01-12 (most recent available as of writing),
# and select a few variables
# The buildable report URL looks like this: 
# https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/daily/start_of_period/county=%2522Alameda%2522,%2522Alpine%2522,%2522Amador%2522,%2522Butte%2522,%2522Calaveras%2522,%2522Colusa%2522,%2522Contra%2520Costa%2522,%2522Del%2520Norte%2522,%2522El%2520Dorado%2522,%2522Fresno%2522,%2522Glenn%2522,%2522Humboldt%2522,%2522Imperial%2522,%2522Inyo%2522,%2522Kern%2522,%2522Kings%2522,%2522Lake%2522,%2522Lassen%2522,%2522Los%2520Angeles%2522,%2522Madera%2522,%2522Marin%2522,%2522Mariposa%2522,%2522Mendocino%2522,%2522Merced%2522,%2522Modoc%2522,%2522Mono%2522,%2522Monterey%2522,%2522Napa%2522,%2522Nevada%2522,%2522Orange%2522,%2522Placer%2522,%2522Plumas%2522,%2522Riverside%2522,%2522Sacramento%2522,%2522San%2520Benito%2522,%2522San%2520Bernardino%2522,%2522San%2520Diego%2522,%2522San%2520Francisco%2522,%2522San%2520Joaquin%2522,%2522San%2520Luis%2520Obispo%2522,%2522San%2520Mateo%2522,%2522Santa%2520Barbara%2522,%2522Santa%2520Clara%2522,%2522Santa%2520Cruz%2522,%2522Shasta%2522,%2522Sierra%2522,%2522Siskiyou%2522,%2522Solano%2522,%2522Sonoma%2522,%2522Stanislaus%2522,%2522Sutter%2522,%2522Tehama%2522,%2522Trinity%2522,%2522Tulare%2522,%2522Tuolumne%2522,%2522Ventura%2522,%2522Yolo%2522,%2522Yuba%2522,%2522UNKNOWN%2522%2520AND%2520network=%2522SNTL%2522%2520AND%2520outServiceDate=%25222100-01-01%2522%257Cname/1991-01-01,2026-12-31/stationId,name,WTEQ::value,SNWD::value,PREC::value,PRCP:-2:value,PRCP::value,TAVG::value,TMAX::value,TMIN::value?fitToScreen=false
# For making more changes, one can go to the main URL, experiment with parameters, and then copy the resulting CSV URL.
# Manually save the file as "manual/snotel/snotel-19910101-20260112.txt"
# and then of course, run the below code to parse and normalize it.
# ============================

# ============================
# SNOTEL — Cell A (FINAL, resolve_out_path)
# - Reads one wide NRCS report export (1991 → present)
# - Melts to long tidy format (station_id × date × variable)
# - Prefers non "-2in" variant when duplicates exist
# - Writes parquet directly into results/ (out_dir)
# ============================

import pandas as pd
import numpy as np
import re
from pathlib import Path

# ---------- config-driven paths ----------
OUT_DIR = Path(CONFIG["out_dir"])
SNOTEL_MANUAL_DIR = Path(resolve_out_path(CONFIG["SNOTEL_MANUAL_DIR"]))         # e.g. "manual/snotel"
RAW_FILE = SNOTEL_MANUAL_DIR / Path(CONFIG["SNOTEL_RAW_FILENAME"]).name
OUT_PARQUET = OUT_DIR / CONFIG["SNOTEL_DAILY_LONG_PARQUET_NAME"]               # e.g. "snotel_daily_long.parquet"

print("SNOTEL Cell A starting")
print("  out_dir   :", OUT_DIR)
print("  raw file  :", RAW_FILE)
print("  out parquet:", OUT_PARQUET)

if not RAW_FILE.exists():
    raise FileNotFoundError(f"Missing SNOTEL raw file: {RAW_FILE}")

# ---------- read raw (robust: skip WCC metadata preamble) ----------
def find_header_row(path: Path, max_lines: int = 500) -> int:
    """
    Find the first line index (0-based) that looks like the actual CSV header.
    WCC report exports often include a preamble before the header.
    """
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        for i, line in enumerate(f):
            if i >= max_lines:
                break
            s = line.strip().lower()
            # header usually starts with 'date' and contains at least one '(' stationId ')'
            if s.startswith("date") and "(" in s and ")" in s and "," in s:
                return i
            # sometimes it's literally just 'date,' without station ids on the same line (rare)
            if s.startswith("date,"):
                return i
    raise ValueError(f"Could not find header row in first {max_lines} lines of {path}")

header_row = find_header_row(RAW_FILE)
print(f"SNOTEL: detected CSV header at line {header_row+1} (1-based)")

df_raw = pd.read_csv(
    RAW_FILE,
    header=header_row,
    low_memory=False,
    encoding="utf-8",
    encoding_errors="replace",
)
# First column is date in this NRCS export
df_raw.rename(columns={df_raw.columns[0]: "date"}, inplace=True)
df_raw["date"] = pd.to_datetime(df_raw["date"], errors="coerce", utc=True)
if df_raw["date"].isna().any():
    bad = df_raw[df_raw["date"].isna()].head(5)
    raise ValueError(f"Found NaT dates after parsing date column; sample:\n{bad}")

# ---------- helper to parse column names ----------
def parse_col(col: str):
    """
    Expected column pattern:
      '<station name> (<stationId>) <variable label...>'
    Returns (station_name, station_id, variable_key, is_minus2_variant) or None
    """
    m = re.match(r"(.+?)\s+\((\d+)\)\s+(.+)", col)
    if not m:
        return None

    station_name = m.group(1).strip()
    station_id = int(m.group(2))
    rest = m.group(3)

    # Canonical variables
    if "Snow Water Equivalent" in rest:
        var = "swe_in"
    elif "Snow Depth" in rest:
        var = "snow_depth_in"
    elif "Precipitation Increment" in rest:
        var = "precip_increment_in"
    elif "Precipitation Accumulation" in rest:
        var = "precip_accum_in"
    elif "Air Temperature Average" in rest:
        var = "tavg_f"
    elif "Air Temperature Maximum" in rest:
        var = "tmax_f"
    elif "Air Temperature Minimum" in rest:
        var = "tmin_f"
    else:
        return None

    is_minus2 = "-2in" in rest
    return station_name, station_id, var, is_minus2

# ---------- melt wide -> long ----------
records = []
for col in df_raw.columns[1:]:
    parsed = parse_col(col)
    if parsed is None:
        continue

    station_name, station_id, var, is_minus2 = parsed

    s = pd.to_numeric(df_raw[col], errors="coerce").astype("float32")

    # Basic validity cleaning (keep conservative; do more later if needed)
    if var.startswith("t"):
        # Fahrenheit: discard extreme negatives (sentinels)
        s = s.where(s > -50)
    else:
        # SWE/Depth/Precip should not be negative
        s = s.where(s >= 0)

    records.append(pd.DataFrame({
        "date": df_raw["date"],
        "station_id": station_id,
        "station_name": station_name,
        "variable": var,
        "value": s,
        "minus2_variant": is_minus2
    }))

if not records:
    raise RuntimeError(
        "Parsed zero SNOTEL data columns. The column naming pattern likely changed. "
        "Inspect df_raw.columns[:50] to update parse_col()."
    )

df_long = pd.concat(records, ignore_index=True)

# ---------- resolve duplicate variants (prefer non '-2in') ----------
df_long.sort_values(
    by=["date", "station_id", "variable", "minus2_variant"],  # False first -> non -2in preferred
    inplace=True
)

df_long = (
    df_long
    .drop_duplicates(subset=["date", "station_id", "variable"], keep="first")
    .drop(columns=["minus2_variant"])
)

df_long["source"] = "SNOTEL"

# ---------- sanity summary ----------
print("SNOTEL Cell A summary")
print("  rows      :", len(df_long))
print("  stations  :", df_long["station_id"].nunique())
print("  variables :", sorted(df_long["variable"].unique()))
print("  date min  :", df_long["date"].min())
print("  date max  :", df_long["date"].max())

# ---------- write (atomic) ----------
tmp_path = OUT_PARQUET.with_suffix(".tmp.parquet")
df_long.to_parquet(tmp_path, index=False)
tmp_path.replace(OUT_PARQUET)

print("Saved:", OUT_PARQUET)

In [ ]:
# ============================
# SNOTEL — Station metadata (AWDB)
# Builds station_id -> lat/lon lookup for gridding
# ============================

import pandas as pd
import numpy as np
import requests
from pathlib import Path
from urllib.parse import quote

OUT_DIR = Path(CONFIG["out_dir"])
SNOTEL_LONG_PATH = OUT_DIR / CONFIG.get("SNOTEL_DAILY_LONG_PARQUET_NAME", "snotel_daily_long.parquet")
META_OUT = OUT_DIR / CONFIG.get("SNOTEL_STATION_META_PARQUET_NAME", "snotel_station_metadata.parquet")

AWDB_URL_TMPL = CONFIG.get(
    "AWDB_META_URL_TEMPLATE",
    "https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/stations?stationTriplets={triplets}&elements="
)

if not SNOTEL_LONG_PATH.exists():
    raise FileNotFoundError(f"Missing SNOTEL long parquet: {SNOTEL_LONG_PATH}")

df_long = pd.read_parquet(SNOTEL_LONG_PATH, columns=["station_id", "station_name"])
station_ids = sorted(df_long["station_id"].dropna().astype(int).unique().tolist())

print("SNOTEL station metadata starting")
print("  stations:", len(station_ids))
print("  out     :", META_OUT)

# If cached, load and only fetch missing
if META_OUT.exists():
    meta = pd.read_parquet(META_OUT)
    have = set(meta["station_id"].astype(int).unique().tolist())
    missing = [sid for sid in station_ids if sid not in have]
    print(f"  cached  : {len(have)} stations | missing: {len(missing)}")
else:
    meta = pd.DataFrame()
    missing = station_ids

def fetch_awdb(triplets: list[str]) -> pd.DataFrame:
    # AWDB expects comma-separated triplets like "1234:CA:SNTL,5678:CA:SNTL"
    trip_str = ",".join(triplets)
    url = AWDB_URL_TMPL.format(triplets=quote(trip_str, safe=",:"))
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    js = r.json()
    # AWDB returns {'stations': [...]} or direct list depending on endpoint version
    rows = js.get("stations", js) if isinstance(js, dict) else js
    return pd.json_normalize(rows)

rows = []
batch_size = 80  # conservative; avoids long URLs
for i in range(0, len(missing), batch_size):
    batch = missing[i:i+batch_size]
    triplets = [f"{sid}:CA:SNTL" for sid in batch]  # CA SNOTEL triplet format
    try:
        got = fetch_awdb(triplets)
    except Exception as e:
        raise RuntimeError(f"AWDB metadata fetch failed for batch {i//batch_size}: {e}")
    rows.append(got)
    if (i//batch_size + 1) % 5 == 0 or (i + batch_size) >= len(missing):
        print(f"  fetched batch {i//batch_size + 1} | total rows so far: {sum(len(x) for x in rows):,}")

new = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

# Normalize / keep only what we need; AWDB fields can vary slightly.
# We key on 'stationTriplet' and/or 'stationId' when present.
if not new.empty:
    # stationId sometimes present; otherwise parse from stationTriplet
    if "stationId" in new.columns:
        new["station_id"] = pd.to_numeric(new["stationId"], errors="coerce")
    elif "stationTriplet" in new.columns:
        new["station_id"] = pd.to_numeric(new["stationTriplet"].str.split(":").str[0], errors="coerce")
    else:
        raise KeyError(f"AWDB response missing stationId/stationTriplet; columns={list(new.columns)[:50]}")

    # lat/lon fields
    lat_col = "latitude" if "latitude" in new.columns else ("lat" if "lat" in new.columns else None)
    lon_col = "longitude" if "longitude" in new.columns else ("lon" if "lon" in new.columns else None)
    if lat_col is None or lon_col is None:
        raise KeyError(f"AWDB response missing lat/lon columns; columns={list(new.columns)[:50]}")

    out = pd.DataFrame({
        "station_id": new["station_id"].astype("Int64"),
        "lat": pd.to_numeric(new[lat_col], errors="coerce"),
        "lon": pd.to_numeric(new[lon_col], errors="coerce"),
        "elev_m": pd.to_numeric(new.get("elevation", np.nan), errors="coerce"),
        "state": new.get("state", "CA"),
        "station_name_meta": new.get("name", pd.NA),
        "station_triplet": new.get("stationTriplet", pd.NA),
    }).dropna(subset=["station_id"])
else:
    out = pd.DataFrame(columns=["station_id","lat","lon","elev_m","state","station_name_meta","station_triplet"])

# Merge with existing cache
if meta.empty:
    meta2 = out
else:
    meta2 = pd.concat([meta, out], ignore_index=True)
    meta2 = meta2.drop_duplicates(subset=["station_id"], keep="last")

# Basic sanity
meta2["station_id"] = meta2["station_id"].astype(int)
ok = meta2.dropna(subset=["lat","lon"])
print("  meta rows:", len(meta2), "| with lat/lon:", len(ok))

tmp = META_OUT.with_suffix(".tmp.parquet")
meta2.to_parquet(tmp, index=False)
tmp.replace(META_OUT)
print("Saved:", META_OUT)

In [ ]:
# ============================
# SNOTEL — Cell B
# Grid stations to 3km grid; write daily shards; stitch final parquet (analysis window)
# ============================

import pandas as pd
import numpy as np
from pathlib import Path
import geopandas as gpd

# ---------- paths ----------
OUT_DIR = Path(CONFIG["out_dir"])

GRID_PATH = Path(resolve_out_path(CONFIG.get("GRID_FILENAME", "grid_3000m_CA_epsg3310.parquet")))
SNOTEL_LONG_PATH = OUT_DIR / CONFIG.get("SNOTEL_DAILY_LONG_PARQUET_NAME", "snotel_daily_long.parquet")

GRIDMAP_PATH = OUT_DIR / CONFIG.get("SNOTEL_GRIDMAP_PARQUET_NAME", "snotel_station_to_grid_3310.parquet")
SHARDS_DIR = OUT_DIR / CONFIG.get("SNOTEL_GRID_SHARDS_DIRNAME", "derived/snotel_grid_shards")
SHARDS_DIR.mkdir(parents=True, exist_ok=True)

start = pd.to_datetime(CONFIG["start_date"]).tz_localize("UTC")
end   = pd.to_datetime(CONFIG["end_date"]).tz_localize("UTC")

final_name_tmpl = CONFIG.get(
    "SNOTEL_DAILY_GRID_PARQUET_NAME",
    "snotel_daily_grid_CA_3000m_epsg3310_{start}_{end}.parquet"
)
FINAL_PATH = OUT_DIR / final_name_tmpl.format(
    start=start.strftime("%Y%m%d"),
    end=end.strftime("%Y%m%d")
)

print("SNOTEL Cell B starting")
print("  Grid     :", GRID_PATH)
print("  SNOTEL A :", SNOTEL_LONG_PATH)
print("  Gridmap  :", GRIDMAP_PATH)
print("  Shards   :", SHARDS_DIR)
print("  FINAL    :", FINAL_PATH)

if not GRID_PATH.exists():
    raise FileNotFoundError(f"Missing grid parquet: {GRID_PATH}")
if not SNOTEL_LONG_PATH.exists():
    raise FileNotFoundError(f"Missing SNOTEL long parquet (Cell A output): {SNOTEL_LONG_PATH}")

# ---------- load grid ----------
ggrid = gpd.read_parquet(GRID_PATH)
if "grid_id" not in ggrid.columns:
    raise KeyError("Grid file must contain grid_id")
if ggrid.crs is None:
    raise ValueError("Grid GeoDataFrame missing CRS")
# ensure EPSG 3310 ops
ggrid = ggrid.to_crs(epsg=int(CONFIG.get("OPS_EPSG", 3310)))

# precompute centroids for nearest mapping
ggrid_cent = ggrid.copy()
ggrid_cent["geometry"] = ggrid_cent.geometry.centroid
ggrid_cent = ggrid_cent[["grid_id", "geometry"]]

# ---------- load SNOTEL long ----------
df = pd.read_parquet(SNOTEL_LONG_PATH)

# Slice to analysis window early (massive speedup)
df = df[(df["date"] >= start) & (df["date"] <= end)].copy()
if df.empty:
    raise ValueError("SNOTEL long table has no rows in analysis window. Check date parsing/timezones.")

META_PATH = OUT_DIR / CONFIG.get("SNOTEL_STATION_META_PARQUET_NAME", "snotel_station_metadata.parquet")
if not META_PATH.exists():
    raise FileNotFoundError(f"Missing station metadata parquet: {META_PATH}. Run the SNOTEL metadata cell first.")

meta = pd.read_parquet(META_PATH)
meta["station_id"] = meta["station_id"].astype(int)

# Build station table from metadata + names from long table
stations = (
    df[["station_id", "station_name"]]
    .drop_duplicates(subset=["station_id"])
    .merge(meta[["station_id","lat","lon"]], on="station_id", how="left")
)

missing_ll = stations["lat"].isna().sum()
if missing_ll:
    print(f"[WARN] {missing_ll} stations missing lat/lon; they will be dropped for gridding.")
stations = stations.dropna(subset=["lat","lon"])


# ---------- station -> grid mapping (cached) ----------
if GRIDMAP_PATH.exists():
    gridmap = pd.read_parquet(GRIDMAP_PATH)
    print(f"Loaded existing gridmap ({len(gridmap):,} stations)")
else:
    gstations = gpd.GeoDataFrame(
        stations,
        geometry=gpd.points_from_xy(stations["lon"], stations["lat"]),
        crs="EPSG:4326"
    ).to_crs(ggrid_cent.crs)

    # nearest grid centroid
    joined = gpd.sjoin_nearest(
        gstations,
        ggrid_cent,
        how="left",
        distance_col="dist_m"
    )

    gridmap = joined[["station_id", "grid_id", "dist_m"]].copy()
    gridmap.to_parquet(GRIDMAP_PATH, index=False)
    print(f"Saved gridmap ({len(gridmap):,} stations) -> {GRIDMAP_PATH}")

# attach grid_id to all records
df = df.merge(gridmap[["station_id", "grid_id"]], on="station_id", how="inner")
if df.empty:
    raise ValueError("After station->grid join, SNOTEL has zero rows. Check station_id alignment.")

# ---------- pick variables of interest (you can expand later) ----------
# Keep everything for now; but when aggregating, we compute grid-level stats per variable.
keep_vars = [
    "precip_increment_in",  # primary precip
    "swe_in",
    "snow_depth_in",
    "tavg_f",
    "tmax_f",
    "tmin_f",
    "precip_accum_in",      # kept but not primary
]
df = df[df["variable"].isin(keep_vars)].copy()

# ---------- aggregate per grid_id x day x variable ----------
# We'll compute mean across stations in same grid cell, plus count.
df["date_day"] = df["date"].dt.floor("D")

all_days = pd.date_range(start=start.floor("D"), end=end.floor("D"), freq="D", tz="UTC")

wrote = skipped = 0
for i, day in enumerate(all_days, start=1):
    day_tag = day.strftime("%Y-%m-%d")
    shard_path = SHARDS_DIR / f"snotel_grid_{day_tag}.parquet"

    if shard_path.exists():
        skipped += 1
        continue

    dday = df[df["date_day"] == day]
    if dday.empty:
        # still write an empty shard to mark completion (resume-safe)
        empty = pd.DataFrame(columns=["grid_id", "date", "variable", "value_mean", "n_stations"])
        empty.to_parquet(shard_path, index=False)
        wrote += 1
        continue

    agg = (
        dday.groupby(["grid_id", "date_day", "variable"], as_index=False)
        .agg(
            value_mean=("value", "mean"),
            n_stations=("value", "count")
        )
        .rename(columns={"date_day": "date"})
    )

    # Optional: pivot to wide per day for easier merges later
    wide = agg.pivot_table(
        index=["grid_id", "date"],
        columns="variable",
        values="value_mean"
    ).reset_index()

    # attach station counts in wide form too
    counts = agg.pivot_table(
        index=["grid_id", "date"],
        columns="variable",
        values="n_stations"
    ).add_prefix("n_").reset_index()

    out = wide.merge(counts, on=["grid_id", "date"], how="left")

    out.to_parquet(shard_path, index=False)
    wrote += 1

    if i % 20 == 0 or i == len(all_days):
        print(f"  day {i}/{len(all_days)} | wrote={wrote} skipped={skipped} | last={day_tag} | rows={len(out):,}")

print("SNOTEL Cell B shards complete")
print("  wrote  :", wrote)
print("  skipped:", skipped)

# ---------- stitch final ----------
shards = sorted(SHARDS_DIR.glob("snotel_grid_*.parquet"))
parts = [pd.read_parquet(p) for p in shards]
final = pd.concat(parts, ignore_index=True)

# Some days may be empty shards; drop them
final = final.dropna(subset=["grid_id", "date"], how="any")

final.to_parquet(FINAL_PATH, index=False)
print("Saved FINAL:", FINAL_PATH, "rows=", len(final))
print(final.head())

In [ ]:
# ============================
# ERA5 MASTER CELL (CDS API) — hardened for ZIP/Nc4 bundles + resume-safe
# - Downloads CA-subset ERA5-Land data into results/{era5_raw,era5_clean}
# - Handles CDS returning ZIP bundles even when requesting "netcdf"
# - Skips downloads if a usable output already exists (either final .nc or existing .zip that has been extracted)
# ============================

from pathlib import Path
import calendar
import time
import zipfile
import cdsapi
import pandas as pd
import xarray as xr

# ---------- toggles (edit these per run) ----------
DO_DAILY_ANALYSIS_WINDOW = True          # 2024-06-01 .. 2024-10-31 (or CONFIG window)
DO_MONTHLY_BASELINE = True               # baseline monthly means (1991-2020 by default)

# If you want to limit daily downloads to only some months, set e.g. [6,7,8,9,10]
DAILY_MONTHS_OVERRIDE = None  # e.g. [6,7,8,9,10] or None to auto from CONFIG start/end

# ---------- helpers ----------
def bbox_to_area(bbox: dict) -> list[float]:
    # CDS expects [North, West, South, East]
    return [float(bbox["nwlat"]), float(bbox["nwlng"]), float(bbox["selat"]), float(bbox["selng"])]

def ensure_dir(p: Path) -> Path:
    p.mkdir(parents=True, exist_ok=True)
    return p

def is_zip(path: Path) -> bool:
    if not path.exists() or path.stat().st_size < 4:
        return False
    head = path.read_bytes()[:4]
    return head.startswith(b"PK\x03\x04") or head.startswith(b"PK\x05\x06") or head.startswith(b"PK\x07\x08")

def looks_like_html(path: Path) -> bool:
    if not path.exists() or path.stat().st_size < 16:
        return False
    head = path.read_bytes()[:64].lstrip().lower()
    return head.startswith(b"<!doctype") or head.startswith(b"<html") or head.startswith(b"{")  # html/json-ish

def month_range_from_dates(d1, d2):
    months = []
    y, m = d1.year, d1.month
    while (y, m) <= (d2.year, d2.month):
        months.append((y, m))
        if m == 12:
            y += 1
            m = 1
        else:
            m += 1
    return months

def retry_sleep(base: float, attempt: int) -> float:
    return base * attempt

def safe_unlink(p: Path) -> None:
    try:
        if p.exists():
            p.unlink()
    except Exception:
        pass

# --- variable aliases (CDS request names vs common ERA5 variable names in NetCDF)
ALIASES = {
    "2m_temperature": ["t2m", "2m_temperature"],
    "snow_depth_water_equivalent": ["sd", "snow_depth_water_equivalent", "sdwe"],
    "volumetric_soil_water_layer_1": ["swvl1", "volumetric_soil_water_layer_1"],
}

def open_nc_quick(path: Path):
    # Keep this minimal to avoid loading actual arrays; just metadata
    return xr.open_dataset(path, engine="netcdf4", decode_times=False)

def validate_era5_netcdf(path: Path, requested_vars: list[str]) -> bool:
    """
    True if path is a readable NetCDF and appears to contain expected structure.
    We do NOT enforce exact variable names (CDS stream files vary), but we do:
    - reject HTML/JSON payloads
    - reject ZIPs masquerading as .nc
    - require: at least one plausible time dim AND lat/lon coordinate presence
    - require: at least one data var that matches one of the requested var aliases (soft check)
    """
    path = Path(path)
    if not path.exists() or path.stat().st_size < 5_000:
        return False
    if is_zip(path) or looks_like_html(path):
        return False
    try:
        ds = open_nc_quick(path)
        # time dim can be time or valid_time
        has_time = ("time" in ds.dims) or ("valid_time" in ds.dims)
        # coord names often latitude/longitude, sometimes lat/lon
        has_latlon = (("latitude" in ds.variables) and ("longitude" in ds.variables)) or (("lat" in ds.variables) and ("lon" in ds.variables))
        if not (has_time and has_latlon):
            return False

        present = set(ds.data_vars.keys()) | set(ds.variables.keys())
        # soft match: at least 1 of requested vars appears via aliases
        want_aliases = []
        for v in requested_vars:
            want_aliases.extend(ALIASES.get(v, [v]))
        if any(a in present for a in want_aliases):
            return True

        # If no alias hits, still allow if dataset has "reasonable" number of data_vars (non-empty)
        return len(ds.data_vars) > 0
    except Exception:
        return False

def choose_best_nc(nc_files: list[Path], requested_vars: list[str]) -> Path:
    """
    From a set of extracted NetCDF files, pick the one most likely to contain the full schema.
    Score: alias hits (heavy) + number of data vars (light).
    """
    want_aliases = []
    for v in requested_vars:
        want_aliases.extend(ALIASES.get(v, [v]))

    best = None
    best_score = -1
    for fp in nc_files:
        try:
            if not validate_era5_netcdf(fp, requested_vars):
                continue
            ds = open_nc_quick(fp)
            present = set(ds.data_vars.keys()) | set(ds.variables.keys())
            alias_hits = sum(1 for a in want_aliases if a in present)
            score = alias_hits * 100 + len(ds.data_vars)
            if score > best_score:
                best_score = score
                best = fp
        except Exception:
            continue
    if best is None:
        raise RuntimeError(f"Could not find a valid NetCDF among extracted files: {[p.name for p in nc_files]}")
    return best

def extract_zip_to_final(zip_path: Path, final_nc: Path, requested_vars: list[str]) -> str:
    """
    Extract zip, select best .nc, write/replace final_nc.
    Keeps the zip in place for provenance/resume.
    """
    zip_path = Path(zip_path)
    final_nc = Path(final_nc)

    if not zip_path.exists():
        raise FileNotFoundError(f"ZIP missing: {zip_path}")

    extract_dir = zip_path.parent / f"_{zip_path.stem}_extracted"
    ensure_dir(extract_dir)

    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_dir)

    nc_files = sorted(extract_dir.glob("*.nc"))
    if not nc_files:
        raise RuntimeError(f"ZIP contained no .nc files: {zip_path.name}")

    best = choose_best_nc(nc_files, requested_vars)

    # Replace final_nc atomically-ish
    tmp = final_nc.with_suffix(".tmp.nc")
    safe_unlink(tmp)
    best.replace(tmp)
    safe_unlink(final_nc)
    tmp.replace(final_nc)

    if not validate_era5_netcdf(final_nc, requested_vars):
        raise RuntimeError(f"Extracted NetCDF failed validation: {final_nc}")

    return "extracted"

# ---------- config ----------
OUT_DIR = Path(CONFIG["out_dir"])
RAW_DIR = ensure_dir(OUT_DIR / CONFIG.get("ERA5_RAW_DIRNAME", "era5_raw"))
CLEAN_DIR = ensure_dir(OUT_DIR / CONFIG.get("ERA5_CLEAN_DIRNAME", "era5_clean"))  # placeholder for later

BBOX = CONFIG["bbox"]
AREA = bbox_to_area(BBOX)

DAILY_DATASET = CONFIG.get("ERA5_DAILY_DATASET", "derived-era5-land-daily-statistics")
MONTHLY_DATASET = CONFIG.get("ERA5_MONTHLY_DATASET", "reanalysis-era5-land-monthly-means")

VARS_DAILY = CONFIG.get("ERA5_VARIABLES_DAILY", [
    "2m_temperature",
    "snow_depth_water_equivalent",
    "volumetric_soil_water_layer_1",
])

MAX_RETRIES = int(CONFIG.get("ERA5_MAX_RETRIES", 6))
BACKOFF_BASE = float(CONFIG.get("ERA5_BACKOFF_BASE_S", 25))
TIMEOUT_S = int(CONFIG.get("ERA5_TIMEOUT_S", 600))

analysis_start = pd.to_datetime(CONFIG["start_date"]).date()
analysis_end   = pd.to_datetime(CONFIG["end_date"]).date()

baseline_start = pd.to_datetime(CONFIG.get("BASELINE_START_DATE", "1991-01-01")).date()
baseline_end   = pd.to_datetime(CONFIG.get("BASELINE_END_DATE", "2020-12-31")).date()

print("ERA5 master cell starting")
print("  out_dir :", OUT_DIR)
print("  RAW_DIR :", RAW_DIR)
print("  CLEAN_DIR (future use):", CLEAN_DIR)
print("  AREA   :", AREA, "(N,W,S,E)")
print("  Daily dataset  :", DAILY_DATASET)
print("  Monthly dataset:", MONTHLY_DATASET)
print("  Daily vars:", VARS_DAILY)
print("  Analysis window :", analysis_start, "→", analysis_end)
print("  Baseline window :", baseline_start, "→", baseline_end)

client = cdsapi.Client()

def cds_download_zip_or_nc(dataset: str, request: dict, final_nc: Path, zip_path: Path) -> str:
    """
    Resume-safe logic:
    - If final_nc exists and validates -> SKIP
    - Else if zip exists -> extract to final_nc (if needed)
    - Else download to zip_path (temporary .part), then extract to final_nc
    """
    final_nc = Path(final_nc)
    zip_path = Path(zip_path)

    # 1) If final output already exists and is valid, skip everything.
    if validate_era5_netcdf(final_nc, VARS_DAILY):
        print(f"[SKIP] {final_nc.name} already present and valid")
        return "skipped"

    # 2) If there's an existing .nc that is actually a ZIP (common if previously saved as .nc), handle it.
    if final_nc.exists() and is_zip(final_nc):
        renamed = final_nc.with_suffix(".zip")
        if not renamed.exists():
            final_nc.rename(renamed)
            print(f"[INFO] Renamed ZIP-masquerading file: {final_nc.name} -> {renamed.name}")
        zip_path = renamed  # prefer the renamed zip going forward

    # 3) If we already have a zip, try extracting (and then validate).
    if zip_path.exists() and is_zip(zip_path):
        print(f"[EXTRACT] {zip_path.name} -> {final_nc.name}")
        return extract_zip_to_final(zip_path, final_nc, VARS_DAILY)

    # 4) Download (to .part first), then decide whether we got zip or nc, then validate/extract.
    tmp = zip_path.with_suffix(zip_path.suffix + ".part")
    safe_unlink(tmp)

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f"[DL] {zip_path.name}  (dataset={dataset})")
            # download into tmp path
            client.retrieve(dataset, request, str(tmp))

            if not tmp.exists() or tmp.stat().st_size < 5_000:
                raise RuntimeError(f"Download produced an empty/small file: {tmp}")

            # move tmp into zip_path (even if it’s actually .nc content; we’ll detect below)
            safe_unlink(zip_path)
            tmp.replace(zip_path)

            # if it's a zip, extract -> final_nc
            if is_zip(zip_path):
                print(f"[OK] downloaded ZIP ({zip_path.stat().st_size/1e6:.1f} MB), extracting...")
                res = extract_zip_to_final(zip_path, final_nc, VARS_DAILY)
                print(f"[OK] {final_nc.name} ready ({final_nc.stat().st_size/1e6:.1f} MB)")
                return "downloaded"

            # if it's not a zip, it might actually be a netcdf payload; rename to final_nc
            if looks_like_html(zip_path):
                raise RuntimeError(f"CDS returned HTML/JSON payload instead of data: {zip_path}")

            # treat as NetCDF-like payload
            safe_unlink(final_nc)
            zip_path.replace(final_nc)

            if not validate_era5_netcdf(final_nc, VARS_DAILY):
                raise RuntimeError(f"Downloaded payload is not a valid ERA5 NetCDF: {final_nc}")

            print(f"[OK] {final_nc.name} ({final_nc.stat().st_size/1e6:.1f} MB)")
            return "downloaded"

        except Exception as e:
            # cleanup tmp if present
            safe_unlink(tmp)
            if attempt == MAX_RETRIES:
                raise
            sleep_s = retry_sleep(BACKOFF_BASE, attempt)
            print(f"[WARN] attempt {attempt}/{MAX_RETRIES} failed: {e}")
            print(f"       sleeping {sleep_s:.0f}s, then retrying...")
            time.sleep(sleep_s)

def download_daily_month(year: int, month: int):
    ndays = calendar.monthrange(year, month)[1]
    request = {
        "variable": VARS_DAILY,
        "year": f"{year:d}",
        "month": f"{month:02d}",
        "day": [f"{d:02d}" for d in range(1, ndays + 1)],
        "daily_statistic": "daily_mean",
        "time_zone": "utc+00:00",
        "frequency": "1_hourly",
        "area": AREA,
        "format": "netcdf",
    }
    final_nc = RAW_DIR / f"era5l_daily_CA_{year}{month:02d}_mean.nc"
    zip_path = RAW_DIR / f"era5l_daily_CA_{year}{month:02d}_mean.zip"
    return cds_download_zip_or_nc(DAILY_DATASET, request, final_nc, zip_path)

def download_monthly_baseline(year: int):
    request = {
        "variable": VARS_DAILY,
        "year": f"{year:d}",
        "month": [f"{m:02d}" for m in range(1, 13)],
        "time": "00:00",
        "area": AREA,
        "format": "netcdf",
    }
    # You already renamed the tripping file to .zip — this logic supports both.
    final_nc = RAW_DIR / f"era5l_monthly_CA_{year}.nc"
    zip_path = RAW_DIR / f"era5l_monthly_CA_{year}.zip"
    return cds_download_zip_or_nc(MONTHLY_DATASET, request, final_nc, zip_path)

# ---------- run daily downloads ----------
if DO_DAILY_ANALYSIS_WINDOW:
    if DAILY_MONTHS_OVERRIDE is not None:
        months = [(analysis_start.year, m) for m in DAILY_MONTHS_OVERRIDE]
    else:
        months = month_range_from_dates(analysis_start, analysis_end)

    print("\nERA5 daily downloads:")
    stats = {"downloaded": 0, "skipped": 0, "extracted": 0}
    for (y, m) in months:
        res = download_daily_month(y, m)
        stats[res] += 1
    print("Daily summary:", stats)

# ---------- run monthly baseline downloads ----------
if DO_MONTHLY_BASELINE:
    print("\nERA5 monthly baseline downloads:")
    stats = {"downloaded": 0, "skipped": 0, "extracted": 0}
    for year in range(baseline_start.year, baseline_end.year + 1):
        res = download_monthly_baseline(year)
        stats[res] += 1
    print("Monthly baseline summary:", stats)

print("\nERA5 master cell complete.")
print("Raw files are in:", RAW_DIR)

In [ ]:
# ============================
# ERA5-Land | Cell A: raw NetCDFs (+ extracted per-variable NCs) -> clean long-format parquet shards
#
# Fixes:
# - Daily top-level .nc may contain only t2m; merges in companion variables from _extracted/*.nc
# - Same for monthly year files if needed
# - Assembly step never hard-fails if no shards exist; it prints diagnostics and skips assembly
#
# Outputs (under results/):
#   - derived/era5_cellA_shards_daily/era5_daily_long_YYYY-MM-DD.parquet
#   - derived/era5_cellA_shards_monthly/era5_monthly_long_YYYY-MM.parquet
#   - era5_daily_long_analysis.parquet    (optional assembly)
#   - era5_monthly_long_baseline.parquet  (optional assembly; OFF by default)
#
# Notes:
# - Cell A only: parse, standardize variable names, convert units (t2m -> degC), write long-format.
# - No grid mapping happens here. Cell B will map lat/lon -> HydroPulse grid and aggregate.
# - Resume-safe: if shard exists, skip; if assembled parquet exists, skip.
# - Reprocessing is manual: delete outputs to rebuild.
# ============================

from pathlib import Path
import re
import numpy as np
import pandas as pd
import xarray as xr

# ----------------------------
# Config + paths
# ----------------------------
OUT_DIR = Path(CONFIG["out_dir"])
RAW_DIR = OUT_DIR / CONFIG.get("ERA5_RAW_DIRNAME", "era5_raw")
assert RAW_DIR.exists(), f"ERA5 RAW_DIR not found: {RAW_DIR}"

analysis_start = pd.to_datetime(CONFIG["start_date"]).date()
analysis_end   = pd.to_datetime(CONFIG["end_date"]).date()

baseline_start = pd.to_datetime(CONFIG.get("BASELINE_START_DATE", "1991-01-01")).date()
baseline_end   = pd.to_datetime(CONFIG.get("BASELINE_END_DATE", "2020-12-31")).date()

VARS = CONFIG.get("ERA5_VARIABLES_DAILY", [
    "2m_temperature",
    "snow_depth_water_equivalent",
    "volumetric_soil_water_layer_1",
])

DERIVED_DIR = OUT_DIR / "derived"
DERIVED_DIR.mkdir(parents=True, exist_ok=True)

DAILY_SHARDS_DIR = DERIVED_DIR / CONFIG.get("ERA5_CELL_A_DAILY_SHARDS_DIRNAME", "era5_cellA_shards_daily")
MONTHLY_SHARDS_DIR = DERIVED_DIR / CONFIG.get("ERA5_CELL_A_MONTHLY_SHARDS_DIRNAME", "era5_cellA_shards_monthly")
DAILY_SHARDS_DIR.mkdir(parents=True, exist_ok=True)
MONTHLY_SHARDS_DIR.mkdir(parents=True, exist_ok=True)

DAILY_LONG_OUT = OUT_DIR / CONFIG.get("ERA5_DAILY_LONG_PARQUET_NAME", "era5_daily_long_analysis.parquet")
MONTHLY_LONG_OUT = OUT_DIR / CONFIG.get("ERA5_MONTHLY_LONG_PARQUET_NAME", "era5_monthly_long_baseline.parquet")

ASSEMBLE_DAILY_ANALYSIS = bool(CONFIG.get("ERA5_ASSEMBLE_DAILY_ANALYSIS", True))
ASSEMBLE_MONTHLY_BASELINE = bool(CONFIG.get("ERA5_ASSEMBLE_MONTHLY_BASELINE", False))

print("ERA5 Cell A starting")
print("  RAW_DIR   :", RAW_DIR)
print("  Analysis  :", analysis_start, "→", analysis_end)
print("  Baseline  :", baseline_start, "→", baseline_end)
print("  VARS      :", VARS)
print("  Daily shards  :", DAILY_SHARDS_DIR)
print("  Monthly shards:", MONTHLY_SHARDS_DIR)
print("  Daily long out :", DAILY_LONG_OUT, "| assemble:", ASSEMBLE_DAILY_ANALYSIS)
print("  Monthly long out:", MONTHLY_LONG_OUT, "| assemble:", ASSEMBLE_MONTHLY_BASELINE)

# ----------------------------
# Variable alias resolution
# ----------------------------
ALIASES = {
    "2m_temperature": ["t2m", "2m_temperature"],
    "snow_depth_water_equivalent": ["sd", "snow_depth_water_equivalent", "sdwe"],
    "volumetric_soil_water_layer_1": ["swvl1", "volumetric_soil_water_layer_1"],
}

def _pick_coord_name(ds, candidates):
    for c in candidates:
        if c in ds.coords or c in ds.variables:
            return c
    return None

def get_time_name(ds):
    return _pick_coord_name(ds, ["time", "valid_time"])

def get_latlon_names(ds):
    lat = _pick_coord_name(ds, ["latitude", "lat"])
    lon = _pick_coord_name(ds, ["longitude", "lon"])
    if lat is None or lon is None:
        raise KeyError(f"Could not find lat/lon coords in dataset. coords={list(ds.coords)} vars={list(ds.variables)[:30]}")
    return lat, lon

def resolve_data_var(ds, canonical_key: str) -> str:
    wants = ALIASES.get(canonical_key, [canonical_key])
    present = set(ds.data_vars.keys())
    for w in wants:
        if w in present:
            return w
    present2 = set(ds.variables.keys())
    for w in wants:
        if w in present2:
            return w
    raise KeyError(
        f"Could not resolve variable '{canonical_key}'. Wants={wants} "
        f"Present(data_vars)={sorted(list(ds.data_vars.keys()))[:50]}"
    )

# ----------------------------
# File discovery
# ----------------------------
def daily_months_from_window(d1, d2):
    out = []
    y, m = d1.year, d1.month
    while (y, m) <= (d2.year, d2.month):
        out.append((y, m))
        if m == 12:
            y, m = y + 1, 1
        else:
            m += 1
    return out

def open_nc(path: Path) -> xr.Dataset:
    # netcdf4 is usually installed in your env; keep it explicit
    return xr.open_dataset(path, engine="netcdf4")

def infer_daily_extracted_dir(year: int, month: int) -> Path:
    return RAW_DIR / f"_era5l_daily_CA_{year}{month:02d}_mean_extracted"

def infer_monthly_extracted_dir(year: int) -> Path:
    return RAW_DIR / f"_era5l_monthly_CA_{year}_extracted"

def find_daily_primary_nc(year: int, month: int) -> Path | None:
    fp = RAW_DIR / f"era5l_daily_CA_{year}{month:02d}_mean.nc"
    return fp if fp.exists() else None

def find_monthly_primary_nc(year: int) -> Path | None:
    fp = RAW_DIR / f"era5l_monthly_CA_{year}.nc"
    return fp if fp.exists() else None

def list_extracted_ncs(extracted_dir: Path) -> list[Path]:
    if not extracted_dir.exists():
        return []
    return sorted([p for p in extracted_dir.glob("*.nc") if p.is_file() and p.stat().st_size > 0])

# ----------------------------
# Build a merged dataset for a period
# ----------------------------
def merge_sources_for_period(primary_nc: Path | None, extracted_dir: Path) -> tuple[xr.Dataset | None, dict]:
    """
    Returns (merged_ds, diag).
    Merges primary_nc + all extracted_dir/*.nc into one Dataset, preferring to include all required vars.
    """
    sources = []
    diag = {"primary": str(primary_nc) if primary_nc else None, "extracted_dir": str(extracted_dir), "extracted_n": 0, "vars_present": []}

    if primary_nc and primary_nc.exists():
        sources.append(primary_nc)

    extracted_ncs = list_extracted_ncs(extracted_dir)
    diag["extracted_n"] = len(extracted_ncs)
    sources.extend(extracted_ncs)

    if not sources:
        return None, diag

    # Open all sources, keep only data vars, then merge
    dsets = []
    for fp in sources:
        try:
            ds = open_nc(fp)
            # Some stream files contain extra meta vars; keep data_vars only
            keep = list(ds.data_vars.keys())
            if keep:
                ds = ds[keep]
            dsets.append(ds)
        except Exception as e:
            print(f"[WARN] failed to open {fp.name}: {e}")

    if not dsets:
        return None, diag

    try:
        merged = xr.merge(dsets, compat="override", join="outer")
    except Exception:
        # fallback: merging can fail on attrs conflicts; strip attrs
        dsets2 = []
        for ds in dsets:
            ds2 = ds.copy()
            ds2.attrs = {}
            for v in ds2.data_vars:
                ds2[v].attrs = {}
            dsets2.append(ds2)
        merged = xr.merge(dsets2, compat="override", join="outer")

    diag["vars_present"] = sorted(list(merged.data_vars.keys()))
    return merged, diag

# ----------------------------
# Shard writer
# ----------------------------
def write_time_slice_shard(ds, tname, latname, lonname, tval, out_path: Path, src_label: str):
    if out_path.exists() and out_path.stat().st_size > 1_000_000:
        print(f"[SKIP] shard exists: {out_path.name} ({out_path.stat().st_size/1e6:.1f} MB)")
        return "skipped"

    dss = ds.sel({tname: tval})

    # Resolve each required variable (fail only if truly absent from merged dataset)
    v_t2m  = resolve_data_var(dss, "2m_temperature")
    v_sdwe = resolve_data_var(dss, "snow_depth_water_equivalent")
    v_swvl1 = resolve_data_var(dss, "volumetric_soil_water_layer_1")

    t2m = dss[v_t2m].values
    sdwe = dss[v_sdwe].values
    swvl1 = dss[v_swvl1].values

    lat = dss[latname].values
    lon = dss[lonname].values

    if lat.ndim == 1 and lon.ndim == 1 and t2m.ndim == 2:
        lon2, lat2 = np.meshgrid(lon, lat)
    else:
        lat2, lon2 = lat, lon

    latf = lat2.reshape(-1).astype(np.float32)
    lonf = lon2.reshape(-1).astype(np.float32)

    t2m_c = (t2m.reshape(-1).astype(np.float32) - 273.15)
    sdwe_f = sdwe.reshape(-1).astype(np.float32)
    swvl1_f = swvl1.reshape(-1).astype(np.float32)

    valid = np.isfinite(latf) & np.isfinite(lonf) & np.isfinite(t2m_c) & np.isfinite(sdwe_f) & np.isfinite(swvl1_f)
    if valid.sum() == 0:
        print(f"[WARN] no valid points for shard {out_path.name}; skipping write.")
        return "empty"

    df = pd.DataFrame({
        "date": pd.to_datetime([pd.to_datetime(tval).date()] * int(valid.sum())),
        "lat": latf[valid],
        "lon": lonf[valid],
        "t2m_c": t2m_c[valid],
        "sdwe": sdwe_f[valid],
        "swvl1": swvl1_f[valid],
        "src_file": src_label,
    })
    df.to_parquet(out_path, index=False)
    print(f"[OK] wrote shard: {out_path.name} rows={len(df):,}")
    return "wrote"

# ----------------------------
# Daily analysis: merge monthly sources, then shard days in window
# ----------------------------
def process_daily_analysis():
    months = daily_months_from_window(analysis_start, analysis_end)
    wrote = skipped = empty = failed_months = 0

    for (y, m) in months:
        primary = find_daily_primary_nc(y, m)
        extracted_dir = infer_daily_extracted_dir(y, m)

        ds, diag = merge_sources_for_period(primary, extracted_dir)
        if ds is None:
            print(f"[WARN] No daily sources found for {y}{m:02d} (primary={primary}, extracted={extracted_dir})")
            failed_months += 1
            continue

        print(f"\n[DAILY] {y}{m:02d} sources:")
        print("  primary :", diag['primary'])
        print("  extracted_dir :", diag['extracted_dir'], f"(n_nc={diag['extracted_n']})")
        print("  merged vars   :", diag["vars_present"][:20], ("..." if len(diag["vars_present"]) > 20 else ""))

        try:
            tname = get_time_name(ds)
            latname, lonname = get_latlon_names(ds)
            times = pd.to_datetime(ds[tname].values)

            # Only the dates in analysis window
            for tval in times:
                d = pd.to_datetime(tval).date()
                if d < analysis_start or d > analysis_end:
                    continue
                out_path = DAILY_SHARDS_DIR / f"era5_daily_long_{d.isoformat()}.parquet"
                res = write_time_slice_shard(ds, tname, latname, lonname, tval, out_path, src_label=f"daily_{y}{m:02d}")
                if res == "wrote":
                    wrote += 1
                elif res == "skipped":
                    skipped += 1
                else:
                    empty += 1

        except Exception as e:
            print(f"[WARN] daily month failed: {y}{m:02d} | {e}")
            failed_months += 1

    print(f"\n[DONE] ERA5 daily shards summary: wrote={wrote} skipped={skipped} empty={empty} failed_months={failed_months}")

    if ASSEMBLE_DAILY_ANALYSIS:
        if DAILY_LONG_OUT.exists() and DAILY_LONG_OUT.stat().st_size > 5_000_000:
            print(f"[SKIP] daily long parquet exists: {DAILY_LONG_OUT} ({DAILY_LONG_OUT.stat().st_size/1e6:.1f} MB)")
        else:
            shard_files = sorted(DAILY_SHARDS_DIR.glob("era5_daily_long_*.parquet"))
            if not shard_files:
                print(f"[WARN] No daily shards found in {DAILY_SHARDS_DIR}; skipping assembly.")
                return
            df = pd.concat([pd.read_parquet(p) for p in shard_files], ignore_index=True)
            df["date"] = pd.to_datetime(df["date"]).dt.normalize()
            df.sort_values(["date", "lat", "lon"], inplace=True)
            df.to_parquet(DAILY_LONG_OUT, index=False)
            print(f"[OK] saved daily long parquet: {DAILY_LONG_OUT} rows={len(df):,}")

# ----------------------------
# Monthly baseline: merge per-year sources, then shard months in baseline window
# ----------------------------
def process_monthly_baseline():
    wrote = skipped = empty = failed_years = 0
    for year in range(baseline_start.year, baseline_end.year + 1):
        primary = find_monthly_primary_nc(year)
        extracted_dir = infer_monthly_extracted_dir(year)

        ds, diag = merge_sources_for_period(primary, extracted_dir)
        if ds is None:
            print(f"[WARN] No monthly sources found for {year} (primary={primary}, extracted={extracted_dir})")
            failed_years += 1
            continue

        print(f"\n[MONTHLY] {year} sources:")
        print("  primary :", diag['primary'])
        print("  extracted_dir :", diag['extracted_dir'], f"(n_nc={diag['extracted_n']})")
        print("  merged vars   :", diag["vars_present"][:20], ("..." if len(diag["vars_present"]) > 20 else ""))

        try:
            tname = get_time_name(ds)
            latname, lonname = get_latlon_names(ds)
            times = pd.to_datetime(ds[tname].values)

            for tval in times:
                d = pd.to_datetime(tval).date()
                if d < baseline_start or d > baseline_end:
                    continue
                ym = f"{d.year:04d}-{d.month:02d}"
                out_path = MONTHLY_SHARDS_DIR / f"era5_monthly_long_{ym}.parquet"
                res = write_time_slice_shard(ds, tname, latname, lonname, tval, out_path, src_label=f"monthly_{year}")
                if res == "wrote":
                    wrote += 1
                elif res == "skipped":
                    skipped += 1
                else:
                    empty += 1

        except Exception as e:
            print(f"[WARN] monthly year failed: {year} | {e}")
            failed_years += 1

    print(f"\n[DONE] ERA5 monthly shards summary: wrote={wrote} skipped={skipped} empty={empty} failed_years={failed_years}")

    if ASSEMBLE_MONTHLY_BASELINE:
        if MONTHLY_LONG_OUT.exists() and MONTHLY_LONG_OUT.stat().st_size > 5_000_000:
            print(f"[SKIP] monthly long parquet exists: {MONTHLY_LONG_OUT} ({MONTHLY_LONG_OUT.stat().st_size/1e6:.1f} MB)")
        else:
            shard_files = sorted(MONTHLY_SHARDS_DIR.glob("era5_monthly_long_*.parquet"))
            if not shard_files:
                print(f"[WARN] No monthly shards found in {MONTHLY_SHARDS_DIR}; skipping assembly.")
                return
            df = pd.concat([pd.read_parquet(p) for p in shard_files], ignore_index=True)
            df["date"] = pd.to_datetime(df["date"]).dt.normalize()
            df.sort_values(["date", "lat", "lon"], inplace=True)
            df.to_parquet(MONTHLY_LONG_OUT, index=False)
            print(f"[OK] saved monthly long parquet: {MONTHLY_LONG_OUT} rows={len(df):,}")

# ----------------------------
# Run
# ----------------------------
process_daily_analysis()
process_monthly_baseline()

print("\nERA5 Cell A complete.")
print("  Daily shards dir  :", DAILY_SHARDS_DIR)
print("  Monthly shards dir:", MONTHLY_SHARDS_DIR)

In [ ]:
# ============================
# ERA5-Land | Cell B: lat/lon points -> HydroPulse grid mapping + daily/monthly gridding shards
#
# Inputs (from Cell A):
#   results/derived/era5_cellA_shards_daily/era5_daily_long_YYYY-MM-DD.parquet
#   results/derived/era5_cellA_shards_monthly/era5_monthly_long_YYYY-MM.parquet
#   results/<GRID_FILENAME>  (GeoParquet w/ grid_id + geometry in EPSG:OPS_EPSG)
#
# Outputs:
#   results/derived/era5_grid_daily_shards/era5_daily_grid_YYYY-MM-DD.parquet
#   results/derived/era5_grid_monthly_shards/era5_monthly_grid_YYYY-MM.parquet
#   results/<ERA5_DAILY_GRID_PARQUET_NAME>      (assembled, analysis window)
#   results/<ERA5_MONTHLY_GRID_PARQUET_NAME>    (assembled, baseline window; optional)
#
# Design:
# - Precompute a stable mapping: (lat, lon) -> grid_id once, cache it to results/
# - Then each shard becomes: read shard -> merge mapping -> groupby grid_id/date -> aggregate
#
# Resume-safe:
# - Mapping parquet reused if present
# - Per-day/per-month grid shards skipped if present
# - Final assembled parquet skipped if present
# - Reprocessing is manual: delete outputs to rebuild
# ============================

from pathlib import Path
import re
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkb

# ----------------------------
# Config + paths
# ----------------------------
OUT_DIR = Path(CONFIG["out_dir"])
DERIVED_DIR = OUT_DIR / "derived"
DERIVED_DIR.mkdir(parents=True, exist_ok=True)

OPS_EPSG = int(CONFIG.get("OPS_EPSG", 3310))

GRID_PATH = Path(resolve_out_path(CONFIG.get("GRID_FILENAME", "grid_3000m_CA_epsg3310.parquet")))

ERA5_DAILY_CELL_A_DIRNAME = CONFIG.get("ERA5_CELL_A_DAILY_SHARDS_DIRNAME", "era5_cellA_shards_daily")
ERA5_MONTHLY_CELL_A_DIRNAME = CONFIG.get("ERA5_CELL_A_MONTHLY_SHARDS_DIRNAME", "era5_cellA_shards_monthly")

CELL_A_DAILY_DIR = DERIVED_DIR / ERA5_DAILY_CELL_A_DIRNAME
CELL_A_MONTHLY_DIR = DERIVED_DIR / ERA5_MONTHLY_CELL_A_DIRNAME

DAILY_GRID_SHARDS_DIR = DERIVED_DIR / CONFIG.get("ERA5_GRID_DAILY_SHARDS_DIRNAME", "era5_grid_daily_shards")
MONTHLY_GRID_SHARDS_DIR = DERIVED_DIR / CONFIG.get("ERA5_GRID_MONTHLY_SHARDS_DIRNAME", "era5_grid_monthly_shards")
DAILY_GRID_SHARDS_DIR.mkdir(parents=True, exist_ok=True)
MONTHLY_GRID_SHARDS_DIR.mkdir(parents=True, exist_ok=True)

ERA5_POINT_GRIDMAP_NAME = CONFIG.get("ERA5_POINT_TO_GRIDMAP_PARQUET_NAME", "era5_point_to_grid_3310.parquet")
GRIDMAP_PATH = OUT_DIR / ERA5_POINT_GRIDMAP_NAME

analysis_start = pd.to_datetime(CONFIG["start_date"]).date()
analysis_end   = pd.to_datetime(CONFIG["end_date"]).date()

baseline_start = pd.to_datetime(CONFIG.get("BASELINE_START_DATE", "1991-01-01")).date()
baseline_end   = pd.to_datetime(CONFIG.get("BASELINE_END_DATE", "2020-12-31")).date()

ERA5_DAILY_GRID_NAME = CONFIG.get(
    "ERA5_DAILY_GRID_PARQUET_NAME",
    f"era5_daily_grid_CA_{CONFIG.get('grid_resolution_m',3000)}m_epsg{OPS_EPSG}_{str(analysis_start).replace('-','')}_{str(analysis_end).replace('-','')}.parquet"
)
ERA5_MONTHLY_GRID_NAME = CONFIG.get(
    "ERA5_MONTHLY_GRID_PARQUET_NAME",
    f"era5_monthly_grid_CA_{CONFIG.get('grid_resolution_m',3000)}m_epsg{OPS_EPSG}_{str(baseline_start).replace('-','')[:4]}01_{str(baseline_end).replace('-','')[:4]}12.parquet"
)

FINAL_DAILY_PATH = OUT_DIR / ERA5_DAILY_GRID_NAME
FINAL_MONTHLY_PATH = OUT_DIR / ERA5_MONTHLY_GRID_NAME

ASSEMBLE_DAILY = bool(CONFIG.get("ERA5_ASSEMBLE_DAILY_GRID", True))
ASSEMBLE_MONTHLY = bool(CONFIG.get("ERA5_ASSEMBLE_MONTHLY_GRID", False))

ERA5_LATLON_ROUND_DECIMALS = int(CONFIG.get("ERA5_LATLON_ROUND_DECIMALS", 6))

print("ERA5 Cell B starting")
print("  GRID_PATH   :", GRID_PATH)
print("  OPS_EPSG    :", OPS_EPSG)
print("  CellA daily :", CELL_A_DAILY_DIR)
print("  CellA monthly:", CELL_A_MONTHLY_DIR)
print("  Gridmap cache:", GRIDMAP_PATH)
print("  Daily grid shards :", DAILY_GRID_SHARDS_DIR)
print("  Monthly grid shards:", MONTHLY_GRID_SHARDS_DIR)
print("  Final daily grid :", FINAL_DAILY_PATH, "| assemble:", ASSEMBLE_DAILY)
print("  Final monthly grid:", FINAL_MONTHLY_PATH, "| assemble:", ASSEMBLE_MONTHLY)
print("  lat/lon rounding decimals:", ERA5_LATLON_ROUND_DECIMALS)

if not GRID_PATH.exists():
    raise FileNotFoundError(f"Missing grid parquet: {GRID_PATH}")

# ----------------------------
# Robust grid loader (handles WKB bytes geometry)
# ----------------------------
def load_grid_geodataframe(grid_path: Path, ops_epsg: int) -> gpd.GeoDataFrame:
    grid = pd.read_parquet(grid_path)

    if "grid_id" not in grid.columns:
        raise KeyError("Grid parquet must include grid_id")
    if "geometry" not in grid.columns:
        raise KeyError("Grid parquet must include geometry column")

    # ensure grid_id is string (your schema is string IDs like CA3310_...)
    grid = grid.copy()
    grid["grid_id"] = grid["grid_id"].astype(str)

    geom = grid["geometry"]
    if len(geom) > 0 and isinstance(geom.iloc[0], (bytes, bytearray, memoryview)):
        grid["geometry"] = grid["geometry"].apply(lambda b: wkb.loads(bytes(b)) if b is not None else None)

    ggrid = gpd.GeoDataFrame(grid, geometry="geometry", crs=f"EPSG:{ops_epsg}").copy()
    ggrid = ggrid[["grid_id", "geometry"]].dropna(subset=["geometry"]).copy()
    return ggrid

ggrid = load_grid_geodataframe(GRID_PATH, OPS_EPSG)
print(f"[OK] Loaded grid: cells={len(ggrid):,}")
print("[DEBUG] grid_id example:", ggrid["grid_id"].iloc[0])

# ----------------------------
# Helpers
# ----------------------------
def list_cellA_shards_daily():
    if not CELL_A_DAILY_DIR.exists():
        return []
    return sorted(CELL_A_DAILY_DIR.glob("era5_daily_long_*.parquet"))

def list_cellA_shards_monthly():
    if not CELL_A_MONTHLY_DIR.exists():
        return []
    return sorted(CELL_A_MONTHLY_DIR.glob("era5_monthly_long_*.parquet"))

def normalize_latlon(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["lat"] = pd.to_numeric(out["lat"], errors="coerce").astype(np.float32)
    out["lon"] = pd.to_numeric(out["lon"], errors="coerce").astype(np.float32)
    out["lat_k"] = out["lat"].round(ERA5_LATLON_ROUND_DECIMALS)
    out["lon_k"] = out["lon"].round(ERA5_LATLON_ROUND_DECIMALS)
    return out

def build_or_load_point_gridmap() -> pd.DataFrame:
    if GRIDMAP_PATH.exists() and GRIDMAP_PATH.stat().st_size > 50_000:
        gm = pd.read_parquet(GRIDMAP_PATH)
        # enforce expected types
        gm["grid_id"] = gm["grid_id"].astype(str)
        print(f"[SKIP] Using existing gridmap: {GRIDMAP_PATH} rows={len(gm):,}")
        return gm

    daily_shards = list_cellA_shards_daily()
    monthly_shards = list_cellA_shards_monthly()

    if daily_shards:
        sample_path = daily_shards[0]
    elif monthly_shards:
        sample_path = monthly_shards[0]
    else:
        raise FileNotFoundError("No ERA5 Cell A shards found (daily or monthly). Run ERA5 Cell A first.")

    print(f"[MAP] Building ERA5 point->grid map from sample shard: {sample_path.name}")
    samp = pd.read_parquet(sample_path, columns=["lat", "lon"])
    samp = normalize_latlon(samp)

    pts = samp[["lat_k", "lon_k"]].drop_duplicates().dropna().copy()
    print(f"[MAP] Unique points: {len(pts):,}")

    gpts = gpd.GeoDataFrame(
        pts,
        geometry=[Point(xy) for xy in zip(pts["lon_k"].astype(float), pts["lat_k"].astype(float))],
        crs="EPSG:4326",
    ).to_crs(epsg=OPS_EPSG)

    joined = gpd.sjoin(gpts, ggrid, how="left", predicate="within")

    missing = joined["grid_id"].isna().sum()
    if missing > 0:
        print(f"[WARN] {missing:,} points not within any grid cell; using nearest join for those.")
        miss = joined[joined["grid_id"].isna()].drop(columns=["grid_id", "index_right"], errors="ignore")
        near = gpd.sjoin_nearest(miss, ggrid, how="left", distance_col="dist_m")
        joined.loc[joined["grid_id"].isna(), "grid_id"] = near["grid_id"].values

    joined = joined.drop(columns=["geometry", "index_right"], errors="ignore")
    gm = joined[["lat_k", "lon_k", "grid_id"]].dropna().copy()

    # Critical fix: grid_id is a STRING in your grid parquet; do not cast to int.
    gm["grid_id"] = gm["grid_id"].astype(str)

    # Atomic-ish write to avoid corrupt cache on failure
    tmp = GRIDMAP_PATH.with_suffix(".tmp.parquet")
    if tmp.exists():
        tmp.unlink()
    gm.to_parquet(tmp, index=False)
    if GRIDMAP_PATH.exists():
        GRIDMAP_PATH.unlink()
    tmp.rename(GRIDMAP_PATH)

    print(f"[OK] Saved gridmap: {GRIDMAP_PATH} rows={len(gm):,}")
    return gm

gridmap = build_or_load_point_gridmap()

# ----------------------------
# Gridding
# ----------------------------
def grid_one_shard(shard_path: Path, out_path: Path, kind: str):
    if out_path.exists() and out_path.stat().st_size > 50_000:
        print(f"[SKIP] Grid shard exists: {out_path.name} ({out_path.stat().st_size/1e6:.1f} MB)")
        return "skipped"

    df = pd.read_parquet(shard_path)
    req = {"date", "lat", "lon", "t2m_c", "sdwe", "swvl1"}
    miss = req - set(df.columns)
    if miss:
        raise KeyError(f"{shard_path.name} missing required columns: {sorted(miss)}")

    df["date"] = pd.to_datetime(df["date"]).dt.normalize()
    df = normalize_latlon(df)

    tmp = df.merge(gridmap, on=["lat_k", "lon_k"], how="left")
    tmp = tmp.dropna(subset=["grid_id"]).copy()
    if tmp.empty:
        print(f"[WARN] After merge, no rows remain for {shard_path.name}; skipping.")
        return "empty"

    tmp["grid_id"] = tmp["grid_id"].astype(str)

    out = (
        tmp.groupby(["grid_id", "date"], as_index=False)
        .agg(
            t2m_c=("t2m_c", "mean"),
            sdwe=("sdwe", "mean"),
            swvl1=("swvl1", "mean"),
            n_points=("t2m_c", "count"),
        )
    )

    out.to_parquet(out_path, index=False)
    print(f"[OK] Wrote {kind} grid shard: {out_path.name} rows={len(out):,} cells={out['grid_id'].nunique():,}")
    return "wrote"

def process_daily_grid():
    shards = list_cellA_shards_daily()
    if not shards:
        print("[WARN] No ERA5 daily Cell A shards found; skipping daily Cell B.")
        return

    wrote = skipped = empty = 0
    for sp in shards:
        m = re.search(r"era5_daily_long_(\d{4}-\d{2}-\d{2})\.parquet$", sp.name)
        if not m:
            continue
        d = pd.to_datetime(m.group(1)).date()
        if d < analysis_start or d > analysis_end:
            continue

        outp = DAILY_GRID_SHARDS_DIR / f"era5_daily_grid_{d.isoformat()}.parquet"
        res = grid_one_shard(sp, outp, kind="daily")
        wrote += (res == "wrote")
        skipped += (res == "skipped")
        empty += (res == "empty")

    print(f"[DONE] ERA5 daily Cell B: wrote={wrote} skipped={skipped} empty={empty}")

    if ASSEMBLE_DAILY:
        if FINAL_DAILY_PATH.exists() and FINAL_DAILY_PATH.stat().st_size > 5_000_000:
            print(f"[SKIP] Final daily grid exists: {FINAL_DAILY_PATH} ({FINAL_DAILY_PATH.stat().st_size/1e6:.1f} MB)")
            return
        files = sorted(DAILY_GRID_SHARDS_DIR.glob("era5_daily_grid_*.parquet"))
        if not files:
            print(f"[WARN] No daily grid shards found in {DAILY_GRID_SHARDS_DIR}; skipping assembly.")
            return
        final = pd.concat([pd.read_parquet(p) for p in files], ignore_index=True)
        final["date"] = pd.to_datetime(final["date"]).dt.normalize()
        final["grid_id"] = final["grid_id"].astype(str)
        final.sort_values(["grid_id", "date"], inplace=True)
        final.to_parquet(FINAL_DAILY_PATH, index=False)
        print(f"[OK] Saved FINAL daily grid: {FINAL_DAILY_PATH} rows={len(final):,} cells={final['grid_id'].nunique():,} dates={final['date'].nunique():,}")

def process_monthly_grid():
    shards = list_cellA_shards_monthly()
    if not shards:
        print("[WARN] No ERA5 monthly Cell A shards found; skipping monthly Cell B.")
        return

    wrote = skipped = empty = 0
    for sp in shards:
        m = re.search(r"era5_monthly_long_(\d{4}-\d{2})\.parquet$", sp.name)
        if not m:
            continue
        ym = m.group(1)
        d = pd.to_datetime(f"{ym}-01").date()
        if d < baseline_start or d > baseline_end:
            continue

        outp = MONTHLY_GRID_SHARDS_DIR / f"era5_monthly_grid_{ym}.parquet"
        res = grid_one_shard(sp, outp, kind="monthly")
        wrote += (res == "wrote")
        skipped += (res == "skipped")
        empty += (res == "empty")

    print(f"[DONE] ERA5 monthly Cell B: wrote={wrote} skipped={skipped} empty={empty}")

    if ASSEMBLE_MONTHLY:
        if FINAL_MONTHLY_PATH.exists() and FINAL_MONTHLY_PATH.stat().st_size > 5_000_000:
            print(f"[SKIP] Final monthly grid exists: {FINAL_MONTHLY_PATH} ({FINAL_MONTHLY_PATH.stat().st_size/1e6:.1f} MB)")
            return
        files = sorted(MONTHLY_GRID_SHARDS_DIR.glob("era5_monthly_grid_*.parquet"))
        if not files:
            print(f"[WARN] No monthly grid shards found in {MONTHLY_GRID_SHARDS_DIR}; skipping assembly.")
            return
        final = pd.concat([pd.read_parquet(p) for p in files], ignore_index=True)
        final["date"] = pd.to_datetime(final["date"]).dt.normalize()
        final["grid_id"] = final["grid_id"].astype(str)
        final.sort_values(["grid_id", "date"], inplace=True)
        final.to_parquet(FINAL_MONTHLY_PATH, index=False)
        print(f"[OK] Saved FINAL monthly grid: {FINAL_MONTHLY_PATH} rows={len(final):,} cells={final['grid_id'].nunique():,} dates={final['date'].nunique():,}")

# ----------------------------
# Run
# ----------------------------
process_daily_grid()
process_monthly_grid()
print("ERA5 Cell B complete.")

In [ ]:
# Next dataset is the USGS-StreamFlow dataset
# ============================
# Base endpoint (Daily Values / DV):
# 	•	https://waterservices.usgs.gov/nwis/dv/

# Key parameters you’ll use (these are the ones that matter):
# 	•	format=rdb (tab-delimited, easy to parse)
# 	•	stateCd=ca (California-only site filter)
# 	•	parameterCd=00060 (discharge, cubic feet per second)
# 	•	siteType=ST (streams)
# 	•	startDT=1991-01-01 (baseline start)
# 	•	endDT=2020-12-31 (baseline end) or endDT=2024-10-31 (analysis end)
# 	•	siteStatus=active (optional)
# 	•	statCd=00003 (daily mean) (supported by DV service outputs)  ￼
# ============================

# URLs look like this: https://waterservices.usgs.gov/nwis/dv/?format=rdb&stateCd=CA&siteType=ST&parameterCd=00060&startDT=1991-01-01&endDT=2025-12-31
# This is a large download! If this fails, then try manually
# downloading from the above URL and saving as "manual/usgs-streamflow/streamflow-19910101-20251231.txt"
# and then run the parsing/normalization cell below.
# ============================


# ============================
# USGS NWIS Daily Values downloader (chunked, resumable)
# Writes to:
#   results/{USGS_RAW_DIRNAME}/  (yearly .rdb chunks + optional combined raw)
#   results/{USGS_CLEAN_DIRNAME}/ (reserved for later)
# ============================

from pathlib import Path
import time
import requests
import pandas as pd

# ---- config ----
OUT_DIR = Path(CONFIG["out_dir"])

RAW_DIR = OUT_DIR / CONFIG.get("USGS_RAW_DIRNAME", "usgs_raw")
CLEAN_DIR = OUT_DIR / CONFIG.get("USGS_CLEAN_DIRNAME", "usgs_clean")
RAW_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

STATE_CD = CONFIG.get("USGS_STATE_CD", "CA")
SITE_TYPE = CONFIG.get("USGS_SITE_TYPE", "ST")
PARAM = CONFIG.get("USGS_PARAM_DISCHARGE", "00060")

START = pd.to_datetime(CONFIG.get("USGS_START_DATE", "1991-01-01")).date()
END   = pd.to_datetime(CONFIG.get("USGS_END_DATE", "2025-12-31")).date()

SLEEP_S = float(CONFIG.get("USGS_SLEEP_S", 2.5))
MAX_RETRIES = int(CONFIG.get("USGS_MAX_RETRIES", 6))
BACKOFF_BASE = float(CONFIG.get("USGS_BACKOFF_BASE_S", 10))
TIMEOUT_S = int(CONFIG.get("USGS_TIMEOUT_S", 180))

BASE_URL = "https://waterservices.usgs.gov/nwis/dv/"

def ok_file(path: Path, min_bytes: int = 50_000) -> bool:
    return path.exists() and path.is_file() and path.stat().st_size >= min_bytes

def fetch_year(year: int) -> Path:
    startDT = f"{year:04d}-01-01"
    endDT   = f"{year:04d}-12-31"
    out = RAW_DIR / f"usgs_dv_{STATE_CD.lower()}_{SITE_TYPE.lower()}_{PARAM}_{year:04d}.rdb"

    if ok_file(out):
        print(f"[SKIP] {out.name} exists ({out.stat().st_size/1e6:.2f} MB)")
        return out

    params = {
        "format": "rdb",
        "stateCd": STATE_CD,
        "siteType": SITE_TYPE,
        "parameterCd": PARAM,
        "startDT": startDT,
        "endDT": endDT,
    }

    headers = {
        "User-Agent": CONFIG.get("USER_AGENT_HEADERS", {}).get("User-Agent", "BlueLeafLabs/HydroPulse"),
        "Accept-Encoding": "gzip, deflate",
    }

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f"[DL] {year} -> {out.name}")
            r = requests.get(BASE_URL, params=params, headers=headers, timeout=TIMEOUT_S)
            r.raise_for_status()

            tmp = out.with_suffix(".tmp")
            tmp.write_bytes(r.content)

            if not ok_file(tmp):
                raise RuntimeError(f"Downloaded file too small for {year}: {tmp.stat().st_size} bytes")

            tmp.replace(out)
            print(f"[OK] {out.name} ({out.stat().st_size/1e6:.2f} MB)")
            time.sleep(SLEEP_S)
            return out

        except Exception as e:
            if attempt == MAX_RETRIES:
                raise
            sleep = BACKOFF_BASE * attempt
            print(f"[WARN] {year} attempt {attempt}/{MAX_RETRIES} failed: {e}")
            print(f"       sleeping {sleep:.0f}s then retrying...")
            time.sleep(sleep)

# ---- run yearly downloads ----
years = list(range(START.year, END.year + 1))

print("USGS download starting (yearly chunks)")
print("  RAW_DIR  :", RAW_DIR)
print("  CLEAN_DIR:", CLEAN_DIR, "(reserved)")
print("  state/site/param:", STATE_CD, SITE_TYPE, PARAM)
print("  date range:", START, "→", END)
print("  years:", years[0], "→", years[-1])

paths = []
for y in years:
    paths.append(fetch_year(y))

print("\nDownloaded/verified chunks:", len(paths))

# ---- optional: combined raw file under RAW_DIR ----
combined = RAW_DIR / f"usgs_dv_{STATE_CD.lower()}_{SITE_TYPE.lower()}_{PARAM}_{START.year:04d}-{END.year:04d}.rdb"
if not ok_file(combined, min_bytes=200_000):
    print("[COMBINE] Writing combined file:", combined.name)
    with combined.open("wb") as w:
        first = True
        for p in paths:
            data = p.read_bytes().splitlines(keepends=True)
            if first:
                w.writelines(data)
                first = False
            else:
                # drop leading comment lines from each subsequent yearly shard
                i = 0
                while i < len(data) and data[i].lstrip().startswith(b"#"):
                    i += 1
                w.writelines(data[i:])
    print("[OK] Combined:", combined.name, f"({combined.stat().st_size/1e6:.2f} MB)")
else:
    print("[SKIP] Combined file already exists:", combined.name)
    


In [ ]:
# ============================
# USGS Cell A (NWIS DV) -> cleaned long parquet (HARDENED + RESUME-SAFE)
# - Robust line parsing, ignore comments, handle repeated headers/spec lines
# - IMPORTANT FIX: parse per-header-block (do NOT build one wide DataFrame for the whole year)
# - Select discharge column per header block using numeric-prefix scoring (handles '14n')
# - Writes per-year shard parquet files so reruns do NOT redo hours of work
# - Assembles final parquet ONLY if final output is missing (manual delete to rebuild)
# ============================

from pathlib import Path
import re
import pandas as pd
import numpy as np

OUT_DIR = Path(CONFIG["out_dir"])
RAW_DIR = OUT_DIR / CONFIG.get("USGS_RAW_DIRNAME", "usgs_raw")
OUT_PARQUET = OUT_DIR / CONFIG.get("USGS_DAILY_LONG_PARQUET_NAME", "usgs_dv_daily_long.parquet")

DERIVED_DIR = OUT_DIR / "derived"
DERIVED_DIR.mkdir(parents=True, exist_ok=True)
YEAR_SHARDS_DIR = DERIVED_DIR / CONFIG.get("USGS_CELL_A_YEAR_SHARDS_DIRNAME", "usgs_cellA_year_shards")
YEAR_SHARDS_DIR.mkdir(parents=True, exist_ok=True)

print("USGS Cell A starting (resume-safe hardened parser, block-wise)")
print("  RAW_DIR      :", RAW_DIR)
print("  OUT_PARQUET  :", OUT_PARQUET)
print("  YEAR_SHARDS  :", YEAR_SHARDS_DIR)

def year_from_filename(name: str) -> int | None:
    m = re.search(r"_(\d{4})\.rdb$", name)
    return int(m.group(1)) if m else None

_num_prefix_re = re.compile(r"^\s*([-+]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?)")

def parse_numeric_prefix_and_suffix(s: str):
    """
    '14n' -> (14.0, 'n')
    '0.08' -> (0.08, NA)
    ''/None -> (NaN, NA)
    """
    if s is None:
        return (np.nan, pd.NA)
    s = str(s).strip()
    if not s:
        return (np.nan, pd.NA)
    m = _num_prefix_re.match(s)
    if not m:
        return (np.nan, pd.NA)
    num = pd.to_numeric(m.group(1), errors="coerce")
    suffix = s[m.end():].strip()
    if suffix == "":
        suffix = pd.NA
    return (float(num) if pd.notna(num) else np.nan, suffix)

def is_spec_line(line: str) -> bool:
    # Typical RDB spec line: "5s\t15s\t20d\t..."
    return bool(re.match(r"^[0-9A-Za-z]+s(\t[0-9A-Za-z]+s)+$", line))

def choose_value_col_for_header(header: list[str], sample_lines: list[str], dt_col: str):
    """
    Choose best discharge col within THIS header.
    Candidates are columns containing '_00060_' excluding *_cd and excluding obvious non-data cols.
    Score candidates by numeric-prefix rate on sample_lines.
    """
    candidates = [c for c in header if ("_00060_" in c) and (not c.endswith("_cd"))]
    if not candidates:
        candidates = [c for c in header if ("00060" in c) and (not c.endswith("_cd")) and (c not in ("agency_cd","site_no",dt_col))]

    if not candidates:
        raise KeyError(f"No discharge candidates in header: {header[:20]} ... (n={len(header)})")

    idx = {c: header.index(c) for c in candidates}
    hlen = len(header)

    def score_col(col: str) -> float:
        j = idx[col]
        good = 0
        tot = 0
        for ln in sample_lines:
            if not ln or ln.startswith("#"):
                continue
            parts = ln.split("\t")
            # normalize width
            if len(parts) < hlen:
                parts = parts + [""] * (hlen - len(parts))
            elif len(parts) > hlen:
                parts = parts[:hlen]
            v = parts[j]
            m = _num_prefix_re.match(str(v).strip())
            tot += 1
            if m:
                good += 1
        return good / tot if tot else 0.0

    best = None
    best_rate = -1.0
    for c in candidates:
        r = score_col(c)
        if r > best_rate:
            best_rate = r
            best = c

    qual_col = (best + "_cd") if (best and (best + "_cd") in header) else None
    return best, qual_col, best_rate, candidates

def parse_rdb_year_blockwise(path: Path):
    """
    Stream parse an RDB file, handling repeated headers.
    Produces standardized rows only (no wide dataframe).
    """
    rows_out = []
    headers_seen = 0
    bad_lines = 0

    header = None
    dt_col = None
    value_col = None
    qual_col = None

    # We use a small buffer after each header to pick the best value column for that header.
    buffer_lines = []
    buffer_target = 300  # small lookahead; cheap but robust

    def flush_buffer_with_chosen_cols():
        nonlocal buffer_lines, value_col, qual_col, bad_lines, rows_out
        if not buffer_lines:
            return
        # process buffered data lines using current header/value_col/qual_col
        hlen = len(header)
        i_ag = header.index("agency_cd") if "agency_cd" in header else None
        i_site = header.index("site_no") if "site_no" in header else None
        i_dt = header.index(dt_col)
        i_val = header.index(value_col) if value_col in header else None
        i_q = header.index(qual_col) if (qual_col and qual_col in header) else None

        for ln in buffer_lines:
            if not ln or ln.startswith("#"):
                continue
            if is_spec_line(ln):
                continue
            parts = ln.split("\t")
            if len(parts) != hlen:
                if len(parts) > hlen:
                    parts = parts[:hlen]
                else:
                    parts = parts + [""] * (hlen - len(parts))
                bad_lines += 1

            # required fields
            if i_site is None or i_val is None:
                continue
            site_no = parts[i_site].strip()
            if not site_no:
                continue

            date_raw = parts[i_dt].strip()
            if not date_raw:
                continue

            v_raw = parts[i_val]
            v_num, v_flag = parse_numeric_prefix_and_suffix(v_raw)

            q_raw = parts[i_q].strip() if i_q is not None else None
            agency = parts[i_ag].strip() if i_ag is not None else None

            rows_out.append({
                "agency_cd": agency,
                "site_no": site_no,
                "date": date_raw,
                "discharge_cfs": v_num,
                "discharge_qual": q_raw if q_raw != "" else None,
                "discharge_flag": v_flag,
                "__source_file": path.name,
            })

        buffer_lines = []

    with open(path, "r", encoding="utf-8", errors="replace") as f:
        for raw in f:
            line = raw.rstrip("\n")
            if not line or line.startswith("#"):
                continue

            # header detected
            if line.startswith("agency_cd\t") and ("\tdatetime\t" in line or "\tdateTime\t" in line):
                # If we were in the middle of a block, flush any buffered lines before switching header
                if header is not None and value_col is not None:
                    flush_buffer_with_chosen_cols()

                header = line.split("\t")
                dt_col = "datetime" if "datetime" in header else "dateTime"
                headers_seen += 1
                value_col = None
                qual_col = None
                buffer_lines = []
                continue

            if header is None:
                continue

            # skip spec lines
            if is_spec_line(line):
                continue

            # if we haven't chosen the discharge column for this header yet, buffer some lines
            if value_col is None:
                buffer_lines.append(line)
                if len(buffer_lines) >= buffer_target:
                    value_col, qual_col, rate, cand = choose_value_col_for_header(header, buffer_lines, dt_col)
                    # now process the buffer
                    flush_buffer_with_chosen_cols()
                continue

            # otherwise process immediately (fast path)
            buffer_lines.append(line)
            if len(buffer_lines) >= 2000:
                flush_buffer_with_chosen_cols()

    # end of file: finalize
    if header is not None and value_col is None and buffer_lines:
        # choose with whatever we have
        value_col, qual_col, rate, cand = choose_value_col_for_header(header, buffer_lines, dt_col)
    if header is not None and value_col is not None:
        flush_buffer_with_chosen_cols()

    df = pd.DataFrame(rows_out)
    return df, headers_seen, bad_lines

def finalize_diagnostics(path: Path):
    df = pd.read_parquet(path)
    df["date"] = pd.to_datetime(df["date"], errors="coerce", utc=True)
    df["discharge_cfs"] = pd.to_numeric(df["discharge_cfs"], errors="coerce")

    null_q = int(df["discharge_cfs"].isna().sum())
    frac_null = null_q / max(1, len(df))

    print("\n[FINALIZE] USGS Cell A output diagnostics")
    print("  rows      :", f"{len(df):,}")
    print("  sites     :", f"{df['site_no'].astype(str).nunique():,}")
    print("  date min  :", df["date"].min())
    print("  date max  :", df["date"].max())
    print("  null discharge_cfs :", f"{null_q:,}", f"({frac_null*100:.2f}%)")
    display(df.head(5))

# --- No rebuild if final exists (your rule) ---
if OUT_PARQUET.exists() and OUT_PARQUET.stat().st_size > 1_000_000:
    print(f"[SKIP] Final output exists ({OUT_PARQUET.stat().st_size/1e6:.1f} MB). Not rebuilding.")
    finalize_diagnostics(OUT_PARQUET)

else:
    if not RAW_DIR.exists():
        raise FileNotFoundError(f"Missing RAW_DIR: {RAW_DIR}")

    in_files = sorted([
        p for p in RAW_DIR.glob("usgs_dv_ca_st_00060_*.rdb")
        if re.search(r"_\d{4}\.rdb$", p.name)
    ])
    if not in_files:
        in_files = sorted([p for p in RAW_DIR.glob("*.rdb") if "1991-2025" not in p.name])
    if not in_files:
        raise FileNotFoundError(f"No per-year .rdb files found under {RAW_DIR}")

    print(f"  Found yearly chunks: {len(in_files)}")
    print("  First/last:", in_files[0].name, "→", in_files[-1].name)

    for fp in in_files:
        year = year_from_filename(fp.name)
        if year is None:
            continue

        shard_path = YEAR_SHARDS_DIR / f"usgs_dv_daily_long_{year}.parquet"
        if shard_path.exists() and shard_path.stat().st_size > 1_000_000:
            print(f"[SKIP] year {year} shard exists: {shard_path.name}")
            continue

        df, headers_seen, bad_lines = parse_rdb_year_blockwise(fp)

        # Normalize date and drop invalids
        df["date"] = pd.to_datetime(df["date"], errors="coerce", utc=True)
        df["site_no"] = df["site_no"].astype(str)

        df = df.dropna(subset=["date", "site_no"]).copy()

        nn = int(pd.to_numeric(df["discharge_cfs"], errors="coerce").notna().sum())
        pct = (nn / max(1, len(df))) * 100.0
        print(f"[OK] year {year} shard: rows={len(df):,} nonnull_discharge={nn:,} ({pct:.2f}%) "
              f"| headers_seen={headers_seen} | ragged_fixed={bad_lines:,}")

        # Fail-fast: if still catastrophically low, stop before burning more time
        if len(df) > 1000 and (nn / len(df)) < 0.05:
            raise RuntimeError(
                f"Year {year} still produced <5% non-null discharge after block-wise parsing. "
                f"This indicates the RDB value strings are not numeric-prefix parsable or the wrong column is being chosen."
            )

        df.to_parquet(shard_path, index=False)

    # Assemble final from shards
    shard_files = sorted(YEAR_SHARDS_DIR.glob("usgs_dv_daily_long_*.parquet"))
    if not shard_files:
        raise RuntimeError("No year shards found to assemble final output.")

    print(f"[ASSEMBLE] Building final from {len(shard_files)} year shards...")
    parts = [pd.read_parquet(p) for p in shard_files]
    out = pd.concat(parts, ignore_index=True).sort_values(["site_no","date"]).reset_index(drop=True)
    out.to_parquet(OUT_PARQUET, index=False)
    print(f"[OK] Wrote final: {OUT_PARQUET} rows={len(out):,}")

    finalize_diagnostics(OUT_PARQUET)

In [ ]:
# ============================
# USGS NWIS DV (daily values) — Cell B (site -> grid mapping + daily gridding shards)
# - Inputs:
#   1) Grid parquet (EPSG:3310) from your earlier grid build
#   2) USGS daily parquet from Cell A (site_no, date, discharge_cfs, ...)
# - Goal:
#   Create a station->grid lookup (site centroids), then produce daily shards for your analysis window,
#   and a final parquet of grid_id x date x discharge.
#
# IMPORTANT:
#   USGS DV downloads do NOT reliably include lat/lon. This cell therefore:
#     A) Tries to use lat/lon if already present in Cell A output, else
#     B) Fetches site metadata (lat/lon) via USGS site service, cached locally, then proceeds.
#
# Resume-safe:
#   - mapping parquet is reused if exists
#   - daily shard files are skipped if already exist
# ============================

from pathlib import Path
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
from shapely.geometry import Point
import time
import re

FORCE_REMAP = True   # True -> rebuild station->grid mapping even if exists
FORCE_SHARDS = True  # True -> rebuild daily shards even if they exist

# ---------- config paths ----------
OUT_DIR = Path(CONFIG["out_dir"])
DERIVED_DIR = OUT_DIR / "derived"
DERIVED_DIR.mkdir(parents=True, exist_ok=True)

GRID_PATH = Path(resolve_out_path(CONFIG.get("GRID_FILENAME", "grid_3000m_CA_epsg3310.parquet")))
USGS_DAILY_PATH = OUT_DIR / CONFIG.get("USGS_DAILY_LONG_PARQUET_NAME", "usgs_dv_daily_long.parquet")

USGS_GRIDMAP_NAME = CONFIG.get("USGS_SITE_TO_GRID_PARQUET_NAME", "usgs_site_to_grid_3310.parquet")
USGS_SITE_META_NAME = CONFIG.get("USGS_SITE_META_PARQUET_NAME", "usgs_site_meta_ca_st.parquet")

GRIDMAP_PATH = OUT_DIR / USGS_GRIDMAP_NAME
SITE_META_PATH = OUT_DIR / USGS_SITE_META_NAME

SHARDS_DIR = DERIVED_DIR / CONFIG.get("USGS_GRID_SHARDS_DIRNAME", "usgs_grid_shards")
SHARDS_DIR.mkdir(parents=True, exist_ok=True)

FINAL_NAME = CONFIG.get(
    "USGS_DAILY_GRID_PARQUET_NAME",
    f"usgs_daily_grid_CA_{CONFIG.get('grid_resolution_m',3000)}m_epsg{int(CONFIG.get('OPS_EPSG',3310))}_{CONFIG['start_date'].replace('-','')}_{CONFIG['end_date'].replace('-','')}.parquet"
)
FINAL_PATH = OUT_DIR / FINAL_NAME

# ---------- CRS ----------
OPS_EPSG = int(CONFIG.get("OPS_EPSG", 3310))

# ---------- time window ----------
analysis_start = pd.to_datetime(CONFIG["start_date"], utc=True).floor("D")
analysis_end   = pd.to_datetime(CONFIG["end_date"],   utc=True).floor("D")

print("USGS Cell B starting")
print("  Grid     :", GRID_PATH)
print("  USGS A   :", USGS_DAILY_PATH)
print("  Site meta:", SITE_META_PATH, "(cache)")
print("  Gridmap  :", GRIDMAP_PATH)
print("  Shards   :", SHARDS_DIR)
print("  FINAL    :", FINAL_PATH)
print("  Config window:", analysis_start.date(), "→", analysis_end.date())

if not GRID_PATH.exists():
    raise FileNotFoundError(f"Missing grid parquet: {GRID_PATH}")
if not USGS_DAILY_PATH.exists():
    raise FileNotFoundError(f"Missing USGS daily parquet (Cell A output): {USGS_DAILY_PATH}")

# ---------- load grid ----------
grid = pd.read_parquet(GRID_PATH)
if "grid_id" not in grid.columns:
    raise KeyError("Grid parquet must include grid_id")
if "geometry" not in grid.columns:
    raise KeyError("Grid parquet must include geometry column (GeoParquet)")

# IMPORTANT: some parquet stores geometry as WKB bytes; coerce if needed
try:
    geom = grid["geometry"]
    if len(geom) and isinstance(geom.iloc[0], (bytes, bytearray, memoryview)):
        ggrid = gpd.GeoDataFrame(grid.copy(), geometry=gpd.GeoSeries.from_wkb(grid["geometry"]), crs=f"EPSG:{OPS_EPSG}")
    else:
        ggrid = gpd.GeoDataFrame(grid.copy(), geometry="geometry", crs=f"EPSG:{OPS_EPSG}")
except Exception:
    ggrid = gpd.GeoDataFrame(grid.copy(), geometry=gpd.GeoSeries.from_wkb(grid["geometry"]), crs=f"EPSG:{OPS_EPSG}")

ggrid = ggrid[["grid_id", "geometry"]].copy()
ggrid["grid_id"] = ggrid["grid_id"].astype(str)

# ---------- load USGS daily ----------
usgs = pd.read_parquet(USGS_DAILY_PATH)

req_cols = {"site_no", "date", "discharge_cfs"}
missing = req_cols - set(usgs.columns)
if missing:
    raise KeyError(f"USGS daily parquet missing required columns: {sorted(missing)}")

usgs["site_no"] = usgs["site_no"].astype(str).str.strip()
usgs["date"] = pd.to_datetime(usgs["date"], utc=True, errors="coerce").dt.floor("D")

if usgs["date"].isna().all():
    raise RuntimeError("All USGS dates parsed to NaT. Check Cell A output 'date' column formatting.")

print("  USGS parquet date span:", usgs["date"].min().date(), "→", usgs["date"].max().date(), f"(rows={len(usgs):,})")

mask = (usgs["date"] >= analysis_start) & (usgs["date"] <= analysis_end)
usgs_win = usgs.loc[mask].copy()

print("  Effective window:", analysis_start.date(), "→", analysis_end.date(), f"(rows={len(usgs_win):,})")

if usgs_win.empty:
    raise RuntimeError(
        "USGS daily data has zero rows in analysis window. "
        "This indicates either CONFIG start/end does not overlap the parquet, or date parsing went wrong."
    )

# ---------- site number normalization ----------
def normalize_site_no(s: str) -> str:
    s = str(s).strip()
    if s.isdigit():
        return s
    digits = re.sub(r"\D+", "", s)
    return digits if digits else s

usgs_win["site_no_norm"] = usgs_win["site_no"].map(normalize_site_no)
sites_needed_norm = sorted(usgs_win["site_no_norm"].dropna().unique().tolist())
print(f"[META] sites needed in window: {len(sites_needed_norm):,}")

# ---------- station metadata acquisition (robust cache merge + coverage checks) ----------
def fetch_usgs_site_meta(site_nos: list[str]) -> pd.DataFrame:
    base = "https://waterservices.usgs.gov/nwis/site/"
    rows = []
    chunk_size = int(CONFIG.get("USGS_SITE_META_CHUNK_SIZE", 250))
    timeout_s = int(CONFIG.get("USGS_TIMEOUT_S", 120))
    sleep_s = float(CONFIG.get("USGS_META_SLEEP_S", 0.2))

    for i in range(0, len(site_nos), chunk_size):
        chunk = site_nos[i:i+chunk_size]
        params = {"format": "rdb", "sites": ",".join(chunk), "siteStatus": "all"}
        r = requests.get(base, params=params, timeout=timeout_s)
        r.raise_for_status()
        lines = r.text.splitlines()

        header_idx = None
        for j, line in enumerate(lines[:800]):
            if not line or line.startswith("#"):
                continue
            if ("site_no" in line) and ("dec_lat_va" in line) and ("dec_long_va" in line):
                header_idx = j
                break
        if header_idx is None:
            print(f"[WARN] site service response missing expected header for chunk starting {chunk[0]}")
            continue

        header = lines[header_idx]
        data_lines = [ln for ln in lines[header_idx+2:] if ln and not ln.startswith("#")]
        if not data_lines:
            continue

        tsv = "\n".join([header] + data_lines)
        df = pd.read_csv(pd.io.common.StringIO(tsv), sep="\t", dtype=str)

        out = pd.DataFrame({
            "site_no": df.get("site_no"),
            "site_name": df.get("station_nm"),
            "lat": pd.to_numeric(df.get("dec_lat_va"), errors="coerce"),
            "lon": pd.to_numeric(df.get("dec_long_va"), errors="coerce"),
        })
        out["site_no"] = out["site_no"].astype(str).str.strip()
        out["site_no_norm"] = out["site_no"].map(normalize_site_no)

        out = out.dropna(subset=["site_no_norm"]).copy()
        rows.append(out)
        time.sleep(sleep_s)

    if not rows:
        return pd.DataFrame(columns=["site_no","site_name","lat","lon","site_no_norm"])

    meta = pd.concat(rows, ignore_index=True)
    meta["site_no_norm"] = meta["site_no_norm"].astype(str).str.strip()
    meta = meta.drop_duplicates(subset=["site_no_norm"])
    return meta

def load_site_meta_cache() -> pd.DataFrame:
    if not SITE_META_PATH.exists():
        return pd.DataFrame(columns=["site_no","site_name","lat","lon","site_no_norm"])
    sm = pd.read_parquet(SITE_META_PATH)

    # backward compat
    if "site_no_norm" not in sm.columns:
        if "site_no" not in sm.columns:
            sm["site_no"] = np.nan
        sm["site_no"] = sm["site_no"].astype(str).str.strip()
        sm["site_no_norm"] = sm["site_no"].map(normalize_site_no)
        sm.to_parquet(SITE_META_PATH, index=False)
        print(f"[META] upgraded cache schema (added site_no_norm): {SITE_META_PATH}")

    for c in ["site_no","site_name","lat","lon","site_no_norm"]:
        if c not in sm.columns:
            sm[c] = np.nan
    sm["site_no_norm"] = sm["site_no_norm"].astype(str).str.strip()
    return sm

site_meta = load_site_meta_cache()

needed_set = set(sites_needed_norm)
have_any = set(site_meta["site_no_norm"].dropna().astype(str))
have_latlon = set(site_meta.loc[site_meta["lat"].notna() & site_meta["lon"].notna(), "site_no_norm"].dropna().astype(str))
print(f"[META] cache coverage: have_any={len(needed_set & have_any):,}/{len(needed_set):,} | have_latlon={len(needed_set & have_latlon):,}/{len(needed_set):,}")

missing_norm = [s for s in sites_needed_norm if s not in have_any]
if missing_norm:
    print(f"[META] fetching missing sites: {len(missing_norm):,}")
    fetched = fetch_usgs_site_meta(missing_norm)

    # append-only; never shrink
    site_meta = pd.concat([site_meta, fetched], ignore_index=True)
    site_meta["site_no_norm"] = site_meta["site_no_norm"].astype(str).str.strip()
    site_meta = site_meta.drop_duplicates(subset=["site_no_norm"])

    site_meta.to_parquet(SITE_META_PATH, index=False)
    print(f"[META] updated cache: {SITE_META_PATH} (rows={len(site_meta):,})")

have_latlon = set(site_meta.loc[site_meta["lat"].notna() & site_meta["lon"].notna(), "site_no_norm"].dropna().astype(str))
print(f"[META] post-fetch coverage: have_latlon={len(needed_set & have_latlon):,}/{len(needed_set):,}")

site_meta_use = site_meta.loc[
    site_meta["site_no_norm"].isin(sites_needed_norm) &
    site_meta["lat"].notna() & site_meta["lon"].notna()
].copy()

if site_meta_use.empty:
    raise RuntimeError("No sites with lat/lon available after metadata acquisition; cannot grid USGS discharge.")

# ---------- build or reuse site->grid map ----------
if GRIDMAP_PATH.exists() and not FORCE_REMAP:
    gridmap = pd.read_parquet(GRIDMAP_PATH)
    print(f"[SKIP] Using existing gridmap: {GRIDMAP_PATH} (rows={len(gridmap):,})")
else:
    gs = gpd.GeoDataFrame(
        site_meta_use.copy(),
        geometry=[Point(xy) for xy in zip(site_meta_use["lon"].astype(float), site_meta_use["lat"].astype(float))],
        crs="EPSG:4326"
    ).to_crs(epsg=OPS_EPSG)

    joined = gpd.sjoin_nearest(
        gs[["site_no_norm","site_no","site_name","lat","lon","geometry"]],
        ggrid[["grid_id","geometry"]],
        how="left",
        distance_col="dist_m"
    )

    gridmap = joined.drop(columns=["geometry"], errors="ignore").copy()
    gridmap["grid_id"] = gridmap["grid_id"].astype(str)
    gridmap["dist_km"] = pd.to_numeric(gridmap.get("dist_m"), errors="coerce") / 1000.0
    gridmap.drop(columns=["dist_m"], inplace=True, errors="ignore")

    gridmap.to_parquet(GRIDMAP_PATH, index=False)
    print(f"[OK] Saved gridmap: {GRIDMAP_PATH} (rows={len(gridmap):,})")

# ---------- build daily shards (site->grid, then aggregate per grid/day) ----------
# Normalize dates *again* on the working frame to guarantee exact equality match vs `days`.
tmp = usgs_win.copy()
tmp["date"] = pd.to_datetime(tmp["date"], utc=True, errors="coerce").dt.floor("D")
tmp["site_no_norm"] = tmp["site_no_norm"].astype(str).str.strip()

tmp = tmp.merge(gridmap[["site_no_norm","grid_id"]], on="site_no_norm", how="inner")
if tmp.empty:
    raise RuntimeError("After joining site->grid, no rows remain (check site_no_norm mapping + gridmap).")

days = pd.date_range(analysis_start, analysis_end, freq="D", tz="UTC")

wrote = 0
skipped = 0

for i, day in enumerate(days, start=1):
    shard = SHARDS_DIR / f"usgs_grid_{day.date().isoformat()}.parquet"
    if shard.exists() and not FORCE_SHARDS:
        skipped += 1
        continue

    ddf = tmp.loc[tmp["date"] == day.floor("D")].copy()
    if ddf.empty:
        skipped += 1
        continue

    out = (
        ddf.groupby(["grid_id","date"], as_index=False)
        .agg(
            discharge_cfs=("discharge_cfs","mean"),
            n_gages=("discharge_cfs","count"),
        )
    )
    out.to_parquet(shard, index=False)
    wrote += 1

    if (i % 20 == 0) or (i == len(days)):
        print(f"  day {i}/{len(days)} | wrote={wrote} skipped={skipped} | last={day.date()} | rows={len(out):,}")

print("USGS Cell B shards complete")
print("  wrote  :", wrote)
print("  skipped:", skipped)

# ---------- combine shards into FINAL parquet ----------
shard_files = sorted(SHARDS_DIR.glob("usgs_grid_*.parquet"))
if not shard_files:
    raise RuntimeError(f"No shard files found in {SHARDS_DIR}. This indicates the shard loop never wrote any day with data.")

final = pd.concat([pd.read_parquet(p) for p in shard_files], ignore_index=True)

final["date"] = pd.to_datetime(final["date"], utc=True).dt.floor("D")
final["grid_id"] = final["grid_id"].astype(str)
final.sort_values(["grid_id","date"], inplace=True)

final.to_parquet(FINAL_PATH, index=False)
print(f"[OK] Saved FINAL: {FINAL_PATH} rows={len(final):,} cells={final['grid_id'].nunique():,} dates={final['date'].nunique():,}")
display(final.head(5))

In [ ]:
# diagnostics: gridmap coverage

from pathlib import Path
import pandas as pd

OUT_DIR = Path(CONFIG["out_dir"])
DERIVED_DIR = OUT_DIR / "derived"

USGS_DAILY_PATH = OUT_DIR / CONFIG.get("USGS_DAILY_LONG_PARQUET_NAME", "usgs_dv_daily_long.parquet")
GRIDMAP_PATH    = OUT_DIR / CONFIG.get("USGS_SITE_TO_GRID_PARQUET_NAME", "usgs_site_to_grid_3310.parquet")

SHARDS_DIR = DERIVED_DIR / CONFIG.get("USGS_GRID_SHARDS_DIRNAME", "usgs_grid_shards")
FINAL_NAME = CONFIG.get(
    "USGS_DAILY_GRID_PARQUET_NAME",
    f"usgs_daily_grid_CA_{CONFIG.get('grid_resolution_m',3000)}m_epsg{int(CONFIG.get('OPS_EPSG',3310))}_{CONFIG['start_date'].replace('-','')}_{CONFIG['end_date'].replace('-','')}.parquet"
)
FINAL_PATH = OUT_DIR / FINAL_NAME

analysis_start = pd.to_datetime(CONFIG["start_date"], utc=True).floor("D")
analysis_end   = pd.to_datetime(CONFIG["end_date"], utc=True).floor("D")

print("[USGS B DIAG] Paths")
print("  OUT_DIR     :", OUT_DIR)
print("  USGS_DAILY  :", USGS_DAILY_PATH, "| exists:", USGS_DAILY_PATH.exists())
print("  GRIDMAP     :", GRIDMAP_PATH, "| exists:", GRIDMAP_PATH.exists())
print("  SHARDS_DIR  :", SHARDS_DIR, "| exists:", SHARDS_DIR.exists())
print("  FINAL_PATH  :", FINAL_PATH, "| exists:", FINAL_PATH.exists())
print("  Window      :", analysis_start.date(), "→", analysis_end.date())

# 1) Confirm window rows exist
usgs = pd.read_parquet(USGS_DAILY_PATH, columns=["site_no", "date", "discharge_cfs"])
usgs["date"] = pd.to_datetime(usgs["date"], utc=True).dt.floor("D")
mask = (usgs["date"] >= analysis_start) & (usgs["date"] <= analysis_end)
usgs_win = usgs.loc[mask].copy()
print("\n[USGS B DIAG] Window rows in Cell A parquet")
print("  rows:", len(usgs_win))
print("  sites:", usgs_win["site_no"].astype(str).str.strip().nunique())

# 2) Verify gridmap integrity
gm = pd.read_parquet(GRIDMAP_PATH)
print("\n[USGS B DIAG] Gridmap columns:", list(gm.columns))
site_key = "site_no_norm" if "site_no_norm" in gm.columns else "site_no"
print("  gridmap rows:", len(gm))
print("  unique sites:", gm[site_key].astype(str).str.strip().nunique(), f"(key={site_key})")
print("  unique grid_id:", gm["grid_id"].nunique())

# 3) Locate any shard files anywhere under derived/
print("\n[USGS B DIAG] Shard discovery")
if DERIVED_DIR.exists():
    all_usgs_shards = sorted(DERIVED_DIR.rglob("usgs_grid_*.parquet"))
else:
    all_usgs_shards = []

print("  derived dir exists:", DERIVED_DIR.exists())
print("  shards matching derived/**/usgs_grid_*.parquet:", len(all_usgs_shards))
if all_usgs_shards:
    print("  first:", all_usgs_shards[0])
    print("  last :", all_usgs_shards[-1])

# 4) If shards exist, show a quick sample read
if all_usgs_shards:
    sample = pd.read_parquet(all_usgs_shards[0])
    print("\n[USGS B DIAG] Sample shard head:")
    print(sample.head())
else:
    print("\n[USGS B DIAG] No shards found anywhere under derived/.")
    print("  That means Cell B never entered (or never completed) the shard-writing loop,")
    print("  or it crashed before writing the first shard.")
    print("  Next step: rerun Cell B and watch for any exception AFTER the gridmap is saved.")

In [ ]:
# MODIS DATA DOWNLOAD
# ============================
# Manual data again
# https://appeears.earthdatacloud.nasa.gov/task/area
# parameters: name: hydropulse_ca_ndvi
# GeoJSON file: results/california_boundary.geojson
# date range: 2000-01-01 to 2020-12-31
# products: MOD13Q1.061 – MODIS/Terra Vegetation Indices 16-Day L3 Global 250m
# From this product, select only: NDVI, VI_Quality, EVI
# Output format: GeoTIFF
# Projection: Native Projection 
# ============================

# Then repeat the same thing for 2024-06-01 to 2024-10-31
# Emails were sent when the jobs were received
# Now to wait for the files to be ready for download

# Files are downloaded as follows:
# results/
# └── manual/
#     └── modis_ndvi_analysis/
#         │   ├── MOD13Q1_*.tif
#         │   └── ...
#     └── modis_ndvi_baseline/
#         │   ├── MOD13Q1_*.tif
#         │   └── ...


# === MODIS NDVI (MOD13Q1) | Cell A: Manual AppEEARS GeoTIFFs -> clean long parquet (analysis window first) ===
# Inputs:
#   results/manual/modis_ndvi_analysis/*.tif  (AppEEARS Area task outputs)
# Output:
#   results/modis_vi_long_analysis.parquet
#   results/derived/modis_cellA_shards_analysis/modis_vi_long_<YYYY-MM-DD>.parquet  (optional shards)
#
# Notes:
# - This is Cell A only: parse, scale, QA-filter, and produce clean long-format records.
# - No grid mapping happens here. Cell B will map pixels -> HydroPulse grid and aggregate per grid cell.
# - Baseline volume is large; this cell targets analysis first via CONFIG["MODIS_ANALYSIS_DIR"].

import re
from pathlib import Path
from datetime import datetime as dt, timedelta
from collections import Counter

import numpy as np
import pandas as pd
import rasterio
from rasterio.errors import RasterioIOError


# ----------------------------
# Core function
# ----------------------------
def run_modis_cellA(
    *,
    tag: str,
    input_dir: Path,
    out_dir: Path,
    assemble_final: bool,
    max_files: int | None = None,
) -> dict:
    """
    MODIS Cell A core processor.
    - Writes per-date shard parquets under: out_dir/derived/modis_cellA_shards_<tag>/
    - Optionally assembles a single long parquet under: out_dir/modis_vi_long_<tag>.parquet
    - Resume-safe:
        - If a shard exists, it is skipped with a diagnostic.
        - If final parquet exists, it is skipped with a diagnostic.
    - Reprocessing is manual: delete outputs to rebuild.
    Returns a dict of diagnostics.
    """

    # ---------- config knobs ----------
    scale_factor = float(CONFIG.get("MODIS_SCALE_FACTOR", 0.0001))

    ENFORCE_QA = bool(CONFIG.get("MODIS_ENFORCE_QA", False))
    ACCEPT_SUMMARY_QA = set(CONFIG.get("MODIS_ACCEPT_SUMMARY_QA", [0]))
    ACCEPT_VI_QUALITY = set(CONFIG.get("MODIS_ACCEPT_VI_QUALITY", []))

    COMMON_FILLS = set(CONFIG.get("MODIS_COMMON_FILL_VALUES", [-3000, -28672, -9999, 32767, 65535]))

    # ---------- paths ----------
    shards_dir = out_dir / "derived" / f"modis_cellA_shards_{tag}"
    shards_dir.mkdir(parents=True, exist_ok=True)

    out_long_parquet = out_dir / f"modis_vi_long_{tag}.parquet"

    print("\nMODIS Cell A starting")
    print("  tag            :", tag)
    print("  input_dir      :", input_dir)
    print("  shards_dir     :", shards_dir)
    print("  assemble_final :", assemble_final)
    print("  out_long       :", out_long_parquet if assemble_final else "(skipped)")
    print("  ENFORCE_QA     :", ENFORCE_QA)

    if not input_dir.exists():
        raise FileNotFoundError(f"MODIS input directory not found: {input_dir}")

    # ---------- helpers ----------
    def parse_modis_date(name: str):
        patterns = [
            r"_doy(\d{4})(\d{3})_",     # AppEEARS canonical
            r"\bA(\d{4})(\d{3})\b",     # MODIS token
            r"doy(\d{4})(\d{3})",       # loose
        ]
        for pat in patterns:
            m = re.search(pat, name)
            if m:
                y = int(m.group(1))
                j = int(m.group(2))
                d = dt(y, 1, 1) + timedelta(days=j - 1)
                return d.date()
        return None

    def parse_feature_id_from_name(name: str):
        m = re.search(r"_(aid\d{4})", name)
        return m.group(1) if m else None

    def parse_var_from_name(name: str):
        upper = name.upper()
        if "SUMMARYQA" in upper or "SUMMARY_QA" in upper:
            return "SummaryQA"
        if "VI_QUALITY" in upper or "VIQUALITY" in upper:
            return "VI_Quality"
        if re.search(r"\bNDVI\b", upper):
            return "NDVI"
        if re.search(r"\bEVI\b", upper):
            return "EVI"
        return None

    def list_geotiffs(root: Path):
        tifs = []
        for ext in ("*.tif", "*.tiff", "*.TIF", "*.TIFF"):
            tifs.extend(root.rglob(ext))
        tifs = [p for p in tifs if p.is_file() and p.stat().st_size > 0]
        tifs = sorted(tifs)
        if max_files is not None:
            tifs = tifs[: int(max_files)]
        return tifs

    def pixel_centers_xy(transform, width: int, height: int):
        cols = np.arange(width) + 0.5
        rows = np.arange(height) + 0.5
        xs = transform.c + transform.a * cols + transform.b * 0.5
        ys = transform.f + transform.e * rows + transform.d * 0.5
        return xs, ys

    def read_single_geotiff_meta(path: Path):
        name = path.name
        date = parse_modis_date(name)
        if date is None:
            return None

        feature_id = parse_feature_id_from_name(name) or CONFIG.get("MODIS_FEATURE_ID")
        var_hint = parse_var_from_name(name)

        try:
            with rasterio.open(path) as src:
                crs_wkt = src.crs.to_wkt() if src.crs else None
                transform = src.transform
                width, height = src.width, src.height
                nodata = src.nodata

                arrays = {}

                # Single-band (one variable per file)
                if src.count == 1:
                    arr = src.read(1)
                    arrays[var_hint or "NDVI"] = arr

                # Multi-band: AppEEARS sometimes packs NDVI/EVI/QA
                elif src.count >= 3:
                    b1, b2, b3 = src.read(1), src.read(2), src.read(3)
                    if var_hint is not None:
                        arrays[var_hint] = b1
                    else:
                        arrays["NDVI"] = b1
                        arrays["EVI"] = b2
                        arrays["VI_Quality"] = b3

                else:
                    b1, b2 = src.read(1), src.read(2)
                    if var_hint is not None:
                        arrays[var_hint] = b1
                    else:
                        arrays["NDVI"] = b1
                        arrays["EVI"] = b2

                return {
                    "date": date,
                    "feature_id": feature_id,
                    "crs_wkt": crs_wkt,
                    "transform": transform,
                    "width": width,
                    "height": height,
                    "nodata": nodata,
                    "arrays": arrays,
                    "name": name,
                }

        except RasterioIOError:
            return None

    def arrays_to_long_df(meta: dict):
        arrays = meta["arrays"]
        if "NDVI" not in arrays:
            return None, "missing_ndvi"

        ndvi_raw = arrays["NDVI"].astype(np.int32, copy=False)
        valid = np.ones(ndvi_raw.shape, dtype=bool)

        nodata = meta["nodata"]
        if nodata is not None:
            valid &= (ndvi_raw != nodata)
        else:
            valid &= ~np.isin(ndvi_raw, list(COMMON_FILLS))

        ndvi = ndvi_raw * scale_factor
        valid &= np.isfinite(ndvi)
        valid &= (ndvi >= -1.0) & (ndvi <= 1.0)

        evi = None
        if "EVI" in arrays:
            evi_raw = arrays["EVI"].astype(np.int32, copy=False)
            if nodata is not None:
                valid &= (evi_raw != nodata)
            else:
                valid &= ~np.isin(evi_raw, list(COMMON_FILLS))
            evi = evi_raw * scale_factor
            valid &= np.isfinite(evi)
            valid &= (evi >= -1.0) & (evi <= 1.0)

        if ENFORCE_QA:
            if "SummaryQA" in arrays:
                sqa = arrays["SummaryQA"].astype(np.int32, copy=False)
                if nodata is not None:
                    valid &= (sqa != nodata)
                if ACCEPT_SUMMARY_QA:
                    valid &= np.isin(sqa, list(ACCEPT_SUMMARY_QA))
            elif "VI_Quality" in arrays:
                viq = arrays["VI_Quality"].astype(np.int32, copy=False)
                if nodata is not None:
                    valid &= (viq != nodata)
                if ACCEPT_VI_QUALITY:
                    valid &= np.isin(viq, list(ACCEPT_VI_QUALITY))

        if valid.sum() == 0:
            return None, "no_valid_pixels_after_filters"

        xs, ys = pixel_centers_xy(meta["transform"], meta["width"], meta["height"])
        rr, cc = np.where(valid)

        df = pd.DataFrame({
            "date": pd.to_datetime([meta["date"]] * len(rr)),
            "feature_id": [meta["feature_id"]] * len(rr),
            "row": rr.astype(np.int32),
            "col": cc.astype(np.int32),
            "x": xs[cc].astype(np.float64),
            "y": ys[rr].astype(np.float64),
            "ndvi": ndvi[rr, cc].astype(np.float32),
            "evi": (evi[rr, cc].astype(np.float32) if evi is not None else np.nan),
            "src_file": meta["name"],
            "crs_wkt": meta["crs_wkt"],
        })
        return df, None

    # ---------- main ----------
    tifs = list_geotiffs(input_dir)
    if len(tifs) == 0:
        raise RuntimeError(f"No GeoTIFFs found under: {input_dir}")

    print(f"[INFO] Found {len(tifs)} GeoTIFFs under: {input_dir}")
    print("[DEBUG] First 10 GeoTIFF filenames:")
    for p in tifs[:10]:
        print("  ", p.name)

    skip_reasons = Counter()
    groups = {}

    for p in tifs:
        meta = read_single_geotiff_meta(p)
        if meta is None:
            if re.search(r"doy\d{7}", p.name, flags=re.IGNORECASE) is None and parse_modis_date(p.name) is None:
                skip_reasons["date_parse_failed"] += 1
            else:
                skip_reasons["read_failed_or_other"] += 1
            continue

        key = (meta["date"], meta["feature_id"])
        if key not in groups:
            groups[key] = {
                "date": meta["date"],
                "feature_id": meta["feature_id"],
                "crs_wkt": meta["crs_wkt"],
                "transform": meta["transform"],
                "width": meta["width"],
                "height": meta["height"],
                "nodata": meta["nodata"],
                "arrays": {},
                "names": [],
            }

        for k, arr in meta["arrays"].items():
            groups[key]["arrays"][k] = arr
        groups[key]["names"].append(meta["name"])

    print(f"[INFO] Parsed into {len(groups)} date-groups")
    if skip_reasons:
        print("[DEBUG] Early skip reasons:", dict(skip_reasons))

    wrote = 0
    skipped = 0
    shard_paths = []

    for (date, fid), g in sorted(groups.items(), key=lambda x: x[0][0].isoformat()):
        shard_path = shards_dir / f"modis_vi_long_{pd.to_datetime(date).date().isoformat()}.parquet"
        if shard_path.exists() and shard_path.stat().st_size > 1_000_000:
            skipped += 1
            shard_paths.append(shard_path)
            print(f"[SKIP] Shard exists: {shard_path.name} ({shard_path.stat().st_size/1e6:.1f} MB)")
            continue

        meta = {
            "date": g["date"],
            "feature_id": g["feature_id"],
            "crs_wkt": g["crs_wkt"],
            "transform": g["transform"],
            "width": g["width"],
            "height": g["height"],
            "nodata": g["nodata"],
            "arrays": g["arrays"],
            "name": "|".join(g["names"])[:500],
        }
        df, reason = arrays_to_long_df(meta)
        if df is None:
            skip_reasons[reason] += 1
            continue

        df.to_parquet(shard_path, index=False)
        wrote += 1
        shard_paths.append(shard_path)
        print(f"[OK] Wrote shard: {shard_path.name} rows={len(df):,}")

    if wrote == 0 and skipped == 0:
        print("[ERROR] No shards produced. Diagnostics:")
        print("  groups:", len(groups))
        print("  skip_reasons:", dict(skip_reasons))
        print("  ENFORCE_QA:", ENFORCE_QA)
        raise RuntimeError("No MODIS shards were produced; check filename parsing, nodata, and QA settings.")

    print(f"[DONE] MODIS Cell A shards: wrote={wrote:,} skipped={skipped:,} dir={shards_dir}")

    if assemble_final:
        if out_long_parquet.exists() and out_long_parquet.stat().st_size > 5_000_000:
            print(f"[SKIP] Long parquet exists: {out_long_parquet} ({out_long_parquet.stat().st_size/1e6:.1f} MB)")
        else:
            # Assemble only from shards we believe exist.
            shard_files = sorted(shards_dir.glob("modis_vi_long_*.parquet"))
            if not shard_files:
                raise RuntimeError(f"No shards found to assemble in {shards_dir}")

            print(f"[INFO] Assembling {len(shard_files)} shards into: {out_long_parquet}")
            dfs = [pd.read_parquet(p) for p in shard_files]
            long_df = pd.concat(dfs, ignore_index=True)
            long_df["date"] = pd.to_datetime(long_df["date"]).dt.normalize()
            long_df = long_df.sort_values(["date", "row", "col"]).reset_index(drop=True)
            long_df.to_parquet(out_long_parquet, index=False)
            print(f"[OK] Saved long parquet: {out_long_parquet} rows={len(long_df):,}")

        # Lightweight preview
        df_preview = pd.read_parquet(out_long_parquet)
        display(df_preview.head(5))
        display(df_preview[["ndvi", "evi"]].describe())

    else:
        # Baseline mode sanity check: read exactly one shard
        shard_files = sorted(shards_dir.glob("modis_vi_long_*.parquet"))
        print("[INFO] assemble_final=False; skipping long parquet by design.")
        if shard_files:
            example = shard_files[0]
            print("  Example shard:", example.name)
            df_preview = pd.read_parquet(example)
            display(df_preview.head(5))
            display(df_preview[["ndvi", "evi"]].describe())

    return {
        "tag": tag,
        "input_dir": str(input_dir),
        "shards_dir": str(shards_dir),
        "out_long_parquet": (str(out_long_parquet) if assemble_final else None),
        "wrote_shards": wrote,
        "skipped_shards": skipped,
        "skip_reasons": dict(skip_reasons),
        "n_groups": len(groups),
    }


# ----------------------------
# Two entrypoints (no manual flag flipping)
# ----------------------------
# delete files from the derived directory if we want to reprocess
def run_modis_analysis():
    out_dir = Path(CONFIG["out_dir"])
    # You can either use CONFIG["MODIS_ANALYSIS_DIR"] (relative) or hard-wire manual/modis_ndvi_analysis.
    # This follows your established convention: out_dir + relative dir from config if present.
    rel = CONFIG.get("MODIS_ANALYSIS_DIR", "manual/modis_ndvi_analysis")
    input_dir = out_dir / rel
    return run_modis_cellA(
        tag="analysis",
        input_dir=input_dir,
        out_dir=out_dir,
        assemble_final=True,
        max_files=None,
    )

# delete files from the derived directory if we want to reprocess
def run_modis_baseline():
    out_dir = Path(CONFIG["out_dir"])
    rel = CONFIG.get("MODIS_BASELINE_DIR", "manual/modis_ndvi_baseline")
    input_dir = out_dir / rel
    return run_modis_cellA(
        tag="baseline",
        input_dir=input_dir,
        out_dir=out_dir,
        assemble_final=False,   # baseline is shards-only to avoid accidental huge concatenation
        max_files=None, # set to a number if we want to test this out
    )


# ----------------------------
# Run (choose which to invoke)
# ----------------------------
# Run analysis (small; assembles final)
diag = run_modis_analysis()

# Run baseline (large; shards-only)
diag = run_modis_baseline()

print("\nMODIS Cell A diagnostics summary:")
print(diag)

In [ ]:
# === HydroPulse | Final Daily Grid Builder (v0: GHCND PRCP only) ===
# Produces: CONFIG["FINAL_DAILY_FILENAME"] as a canonical daily grid table:
#   grid_id × date → prcp_mm + QC
#
# Inputs:
#   - Grid parquet (EPSG:3310): CONFIG["GRID_FILENAME"]
#   - GHCND cleaned station-day parquet: resolve_out_path(CONFIG["GHCND_CLEAN_PARQUET_NAME"])
#
# Notes for later HeatShield refactor:
#   - This cell establishes the common "final builder" contract: {grid_id, date, variables..., QC...}
#   - Keep the interface stable; only swap/extend source adapters per repo.

import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from sklearn.neighbors import BallTree

# -----------------------------
# Paths
# -----------------------------
OUT_DIR = Path(CONFIG["out_dir"])
GRID_PATH = Path(resolve_out_path(CONFIG["GRID_FILENAME"]))
GHCND_PATH = Path(resolve_out_path(CONFIG["GHCND_CLEAN_PARQUET_NAME"]))
FINAL_PATH = Path(resolve_out_path(CONFIG["FINAL_DAILY_FILENAME"]))

# Resume via daily shards
SHARDS_DIR = OUT_DIR / "derived" / "final_daily_shards_prcp"
SHARDS_DIR.mkdir(parents=True, exist_ok=True)

print("GRID_PATH:", GRID_PATH)
print("GHCND_PATH:", GHCND_PATH)
print("SHARDS_DIR:", SHARDS_DIR)
print("FINAL_PATH:", FINAL_PATH)

# -----------------------------
# Column contract (confirmed)
# -----------------------------
STATION_COL = "station_core"
DATE_COL    = "date"
LAT_COL     = "lat"
LON_COL     = "lon"
PRCP_COL_IN = "precipitation_mm"

# Output column naming for the final table
PRCP_COL_OUT = "prcp_mm"

# -----------------------------
# Tunables (can later move to config)
# -----------------------------
K = int(CONFIG.get("PRCP_IDW_K", 8))                        # k nearest stations
HARD_CAP_KM = float(CONFIG.get("PRCP_HARD_CAP_KM", 100.0))  # ignore stations beyond this radius
POWER = float(CONFIG.get("PRCP_IDW_POWER", 2.0))            # IDW power

OPS_EPSG = int(CONFIG.get("OPS_EPSG", 3310))
WGS84_EPSG = int(CONFIG.get("WGS84_EPSG", 4326))

# -----------------------------
# Helpers (keep these stable across repos)
# -----------------------------
def ensure_epsg(gdf: gpd.GeoDataFrame, epsg: int) -> gpd.GeoDataFrame:
    if gdf.crs is None:
        gdf = gdf.set_crs(f"EPSG:{WGS84_EPSG}")
    if (gdf.crs.to_epsg() or 0) != epsg:
        gdf = gdf.to_crs(epsg)
    return gdf

def ensure_grid_id(grid: gpd.GeoDataFrame) -> tuple[gpd.GeoDataFrame, str]:
    for c in ["grid_id", "cell_id", "id"]:
        if c in grid.columns:
            return grid, c
    grid = grid.copy()
    grid["grid_id"] = np.arange(len(grid), dtype=np.int32)
    return grid, "grid_id"

def build_balltree_from_points(geom: gpd.GeoSeries) -> BallTree:
    xy = np.column_stack([geom.x.values, geom.y.values])
    return BallTree(xy, metric="euclidean")

def idw(dist_m: np.ndarray, vals: np.ndarray, power: float) -> float:
    if np.any(dist_m == 0):
        return float(vals[np.argmin(dist_m)])
    w = 1.0 / np.power(dist_m, power)
    return float(np.sum(w * vals) / np.sum(w))

def interpolate_prcp_for_day(grid_centroids: gpd.GeoSeries, stations_day: gpd.GeoDataFrame) -> pd.DataFrame:
    """
    Returns a DF aligned to grid_centroids order with columns:
      prcp_mm, n_used, maxdist_km, method
    """
    n_cells = len(grid_centroids)
    out = pd.DataFrame({
        PRCP_COL_OUT: np.full(n_cells, np.nan, dtype=float),
        "n_used": np.zeros(n_cells, dtype=np.int16),
        "maxdist_km": np.full(n_cells, np.nan, dtype=float),
        "method": np.full(n_cells, None, dtype=object),
    })

    if stations_day is None or len(stations_day) == 0:
        return out

    # Build tree on station points
    tree = build_balltree_from_points(stations_day.geometry)
    vals = stations_day[PRCP_COL_IN].to_numpy(dtype=float)

    qxy = np.column_stack([grid_centroids.x.values, grid_centroids.y.values])
    k_eff = min(K, len(stations_day))
    dist_m, idx = tree.query(qxy, k=k_eff)

    hard_cap_m = HARD_CAP_KM * 1000.0

    for i in range(n_cells):
        d = dist_m[i]
        j = idx[i]

        # Apply hard cap
        mask = d <= hard_cap_m
        if not np.any(mask):
            continue

        d_use = d[mask]
        v_use = vals[j[mask]]

        # Drop NaNs (defensive)
        good = np.isfinite(v_use)
        d_use = d_use[good]
        v_use = v_use[good]
        if len(v_use) == 0:
            continue

        if len(v_use) == 1:
            out.at[i, PRCP_COL_OUT] = float(v_use[0])
            out.at[i, "method"] = "nearest"
        else:
            out.at[i, PRCP_COL_OUT] = idw(d_use, v_use, power=POWER)
            out.at[i, "method"] = f"idw_k{len(v_use)}"

        out.at[i, "n_used"] = int(len(v_use))
        out.at[i, "maxdist_km"] = float(np.max(d_use) / 1000.0)

    return out

# -----------------------------
# Load grid (EPSG:3310) and compute centroids
# -----------------------------
grid = gpd.read_parquet(GRID_PATH)
grid = ensure_epsg(grid, OPS_EPSG)
grid, GID_COL = ensure_grid_id(grid)
centroids = grid.geometry.centroid

print(f"Grid: {len(grid)} cells | CRS EPSG: {grid.crs.to_epsg()} | id col: {GID_COL}")

# -----------------------------
# Load cleaned GHCND station-day table
# -----------------------------
cdo = pd.read_parquet(GHCND_PATH)

required = [STATION_COL, DATE_COL, LAT_COL, LON_COL, PRCP_COL_IN]
missing = [c for c in required if c not in cdo.columns]
if missing:
    raise KeyError(f"Missing required columns in {GHCND_PATH.name}: {missing}. Have: {list(cdo.columns)}")

# Normalize date to UTC day
cdo[DATE_COL] = pd.to_datetime(cdo[DATE_COL], utc=True, errors="coerce").dt.normalize()

# Filter date window
start = pd.to_datetime(CONFIG["start_date"], utc=True).normalize()
end = pd.to_datetime(CONFIG["end_date"], utc=True).normalize()
cdo = cdo[(cdo[DATE_COL] >= start) & (cdo[DATE_COL] <= end)].copy()

# Drop invalid coords / missing precip
cdo = cdo[np.isfinite(cdo[LAT_COL]) & np.isfinite(cdo[LON_COL])].copy()
cdo = cdo[np.isfinite(cdo[PRCP_COL_IN])].copy()

print("Station-day rows:", len(cdo), "| stations:", cdo[STATION_COL].nunique(), "| dates:", cdo[DATE_COL].nunique())

# GeoDataFrame in OPS_EPSG
pts = gpd.GeoDataFrame(
    cdo,
    geometry=gpd.points_from_xy(cdo[LON_COL], cdo[LAT_COL]),
    crs=f"EPSG:{WGS84_EPSG}"
)
pts = ensure_epsg(pts, OPS_EPSG)

# Pre-group by date for speed (avoid repeated boolean filters)
pts_by_date = {d: df for d, df in pts.groupby(DATE_COL)}
all_dates = pd.date_range(start=start, end=end, freq="D")

# -----------------------------
# Daily loop with resume
# -----------------------------
written = 0
skipped = 0

for d in all_dates:
    tag = d.strftime("%Y%m%d")
    shard_path = SHARDS_DIR / f"final_prcp_{tag}.parquet"
    if shard_path.exists():
        skipped += 1
        continue

    day_pts = pts_by_date.get(d)
    interp = interpolate_prcp_for_day(centroids, day_pts)

    out = pd.DataFrame({
        GID_COL: grid[GID_COL].values,
        "date": np.full(len(grid), d),
    })
    out = pd.concat([out, interp], axis=1)

    out.to_parquet(shard_path, index=False)
    written += 1

print(f"Shards written: {written} | skipped (resume): {skipped} | total days: {len(all_dates)}")

# -----------------------------
# Compose FINAL parquet
# -----------------------------
shards = sorted(SHARDS_DIR.glob("final_prcp_*.parquet"))
if not shards:
    raise FileNotFoundError(f"No shards found in {SHARDS_DIR}")

df_final = pd.concat((pd.read_parquet(p) for p in shards), ignore_index=True)

df_final["date"] = pd.to_datetime(df_final["date"], utc=True).dt.normalize()
df_final = df_final.sort_values([GID_COL, "date"]).reset_index(drop=True)

FINAL_PATH.parent.mkdir(parents=True, exist_ok=True)
df_final.to_parquet(FINAL_PATH, index=False)

print("Saved FINAL:", FINAL_PATH)
print("Final rows:", len(df_final), "| cells:", df_final[GID_COL].nunique(), "| dates:", df_final["date"].nunique())
print(df_final.head())